In [ ]:
"""
I3D Dataset Processor v8.0 - Scientific Data Ready
===================================================
Production-ready processor. Deterministic for cached files only.

Key Features:
- Fixed audio features: 94 (not 108)
- Deterministic for cached files (transcript + translation frozen)
- Non-deterministic for new files (external APIs: Google STT + Translate)
- Translation cache (MD5-based)
- Fixed audio enhancement parameters
- Per-file decision logging
- Data dictionary generation
- Run manifest for reproducibility
- Explicit librosa parameters
- MediaPipe: 478 face (468+10 iris) + 33 pose landmarks (validated at runtime)

Author: [Your Name]
Version: 8.0
Date: 2025-01-30
"""


import os
import re
import csv
import sys
import time
import json
import yaml
import subprocess
import pickle
import hashlib
import platform
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional

# Data processing
import numpy as np
import pandas as pd

# Audio processing
import wave
import librosa
import speech_recognition as sr
from pydub import AudioSegment

# Computer Vision
import cv2
import mediapipe as mp

# NLP
import nltk

# Translation
from deep_translator import GoogleTranslator

# Visualization
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend
import matplotlib.pyplot as plt
import seaborn as sns

# Progress bar
from tqdm import tqdm

# Logging
import logging

# Suppress only specific warnings
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning, module='librosa')

# Download NLTK resources (should be pre-installed)
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    logging.error("❌ NLTK 'punkt' tokenizer not found!")
    logging.error("   Please install: python -m nltk.downloader punkt")
    logging.error("   Or run in Python: import nltk; nltk.download('punkt')")
    raise RuntimeError(
        "NLTK 'punkt' not found. Install with: python -m nltk.downloader punkt"
    )


# Initialize MediaPipe
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose

# Version info
VERSION = "8.0"
PIPELINE_NAME = "I3D-Processor-ScientificData"


# ==================== MANUAL TRANSCRIPTS ====================
MANUAL_TRANSCRIPTS = {
    'LIE_Bataknese_07_Female_G_I_A_1.mov': 'tiga',
    'LIE_Bataknese_11_Male_NG_S_A_1.mov': 'dua',
    'LIE_Bataknese_18_Male_NG_S_A_2.mov': 'kalo saya pikir ini ada dua pelaku ini pak, ada satu yang mengahantar, tapi dak kelihatan mungkin yang mengantar itu',
    'LIE_Bataknese_19_Male_NG_C_A_2.mov': 'rame',
    'LIE_Bataknese_24_Female_G_I_B_1.mov': 'dua',
    'LIE_Bataknese_27_Male_NG_C_B_1.mov': 'dua',
    'LIE_Bataknese_27_Male_NG_C_B_2.mov': 'ramai',
    'LIE_BugisMakassarese_07_Male_G_C_A_1.mov': 'tiga',
    'LIE_BugisMakassarese_07_Male_G_C_A_2.mov': 'ee sepi ndak ado orang',
    'LIE_BugisMakassarese_09_Female_G_D_A_1.mov': 'dua',
    'LIE_BugisMakassarese_10_Female_G_I_A_2.mov': 'em rame',
    'LIE_BugisMakassarese_21_Male_G_D_A_1.mov': 'dua',
    'LIE_BugisMakassarese_22_Female_NG_S_A_2.mov': 'ramai',
    'LIE_BugisMakassarese_26_Female_G_C_B_1.mov': 'tiga',
    'LIE_BugisMakassarese_28_Male_NG_C_B_2.mov': 'eeh ramai',
    'LIE_Javanese_02_Female_NG_S_A_2.mov': 'sepi',
    'LIE_Javanese_14_Female_NG_C_B_1.mov': 'dua',
    'LIE_Javanese_20_Female_G_D_B_1.mov': 'lima',
    'LIE_Javanese_20_Female_G_D_B_2.mov': 'rame',
    'LIE_Javanese_26_Female_G_C_A_1.mov': 'tiga',
    'LIE_Madurese_04_Male_NG_C_A_2.mov': 'hmm',
    'LIE_Madurese_05_Female_G_D_B_1.mov': 'dua',
    'LIE_Madurese_20_Male_G_I_B_1.mov': 'dua',
    'LIE_Madurese_20_Male_G_I_B_2.mov': 'ramai',
    'LIE_Madurese_28_Female_G_I_A_2.mov': 'sepi',
    'LIE_Madurese_43_Female_G_D_B_1.mov': 'dua',
    'LIE_Sundanese_16_Female_NG_C_A_4.mov': 'pada malam hari ada dua orang pelaku satu permpuan dan satu laki laki memasuki rumah dan mengambil sebuah gas satu',
    'TRUTH_BugisMakassarese_21_Male_G_D_A_2.mov': 'sepi',
    'TRUTH_BugisMakassarese_23_Female_NG_S_A_2.mov': 'sepi',
    'TRUTH_Javanese_02_Female_NG_S_A_4.mov': 'dua',
    'TRUTH_Javanese_06_Female_G_D_A_4.mov': 'dua',
    'TRUTH_Javanese_14_Female_NG_C_B_4.mov': 'dua',
    'TRUTH_Javanese_20_Female_G_D_B_4.mov': 'dua',
    'TRUTH_Sundanese_12_Female_G_C_A_2.mov': 'suasana tempat kejadian itu saat itu, rumahnya kosong. Jadi memang hanya ada satu pelaku itu saja'
}


# ==================== ENVIRONMENT CAPTURE ====================
def capture_environment() -> Dict:
    """Capture complete environment for reproducibility"""
    import pkg_resources
    
    env_info = {
        'timestamp': datetime.now().isoformat(),
        'pipeline_version': VERSION,
        'pipeline_name': PIPELINE_NAME,
        'python_version': sys.version,
        'platform': {
            'system': platform.system(),
            'release': platform.release(),
            'version': platform.version(),
            'machine': platform.machine(),
            'processor': platform.processor()
        },
        'packages': {}
    }
    
    # Key packages
    key_packages = [
        'numpy', 'pandas', 'librosa', 'opencv-python', 'mediapipe',
        'SpeechRecognition', 'pydub', 'deep-translator', 'nltk',
        'matplotlib', 'seaborn', 'tqdm', 'pyyaml'
    ]
    
    for pkg in key_packages:
        try:
            version = pkg_resources.get_distribution(pkg).version
            env_info['packages'][pkg] = version
        except:
            env_info['packages'][pkg] = 'not_found'
    
    # FFmpeg version
    try:
        result = subprocess.run(['ffmpeg', '-version'], 
                              capture_output=True, text=True, timeout=5)
        ffmpeg_version = result.stdout.split('\n')[0]
        env_info['ffmpeg_version'] = ffmpeg_version
    except:
        env_info['ffmpeg_version'] = 'not_found'
    
    return env_info


# ==================== TRANSCRIPT CACHE MANAGER ====================
class TranscriptCacheManager:
    """Manage frozen transcripts for deterministic results"""
    
    def __init__(self, cache_dir: Path):
        self.cache_dir = cache_dir
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.transcripts_dir = self.cache_dir / 'transcripts'
        self.transcripts_dir.mkdir(parents=True, exist_ok=True)
    
    def get_transcript_path(self, filename: str) -> Path:
        """Get path to transcript JSON file"""
        video_stem = Path(filename).stem
        return self.transcripts_dir / f"{video_stem}.json"
    
    def load_transcript(self, filename: str) -> Optional[Dict]:
        """Load cached transcript"""
        transcript_path = self.get_transcript_path(filename)
        
        if not transcript_path.exists():
            return None
        
        try:
            with open(transcript_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return data
        except Exception as e:
            logging.error(f"Failed to load transcript cache for {filename}: {str(e)}")
            return None
    
    def save_transcript(self, filename: str, text: str, source: str, 
                       attempt: int = 0, mode: str = 'normal', 
                       audio_dbfs: float = 0.0, 
                       stt_energy_threshold: float = 0.0,
                       enhancement_params: Dict = None):
        """Save transcript to cache"""
        transcript_path = self.get_transcript_path(filename)
        
        data = {
            'filename': filename,
            'text': text,
            'source': source,  # 'manual', 'stt', 'stt_ultra'
            'attempt': attempt,
            'mode': mode,  # 'normal', 'ultra'
            'audio_dbfs': audio_dbfs,
            'stt_energy_threshold': stt_energy_threshold,
            'enhancement_params': enhancement_params or {},
            'timestamp': datetime.now().isoformat(),
            'version': VERSION
        }
        
        try:
            with open(transcript_path, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)
            return True
        except Exception as e:
            logging.error(f"Failed to save transcript cache for {filename}: {str(e)}")
            return False
    
    def exists(self, filename: str) -> bool:
        """Check if transcript cache exists"""
        return self.get_transcript_path(filename).exists()


# ==================== TRANSLATION CACHE MANAGER ====================
class TranslationCacheManager:
    """Manage translation cache for deterministic results"""
    
    def __init__(self, cache_dir: Path):
        self.cache_dir = cache_dir
        self.cache_dir.mkdir(parents=True, exist_ok=True)
        self.cache_file = self.cache_dir / 'translation_cache.json'
        self.cache = self._load_cache()
    
    def _load_cache(self) -> Dict:
        """Load translation cache from file"""
        if not self.cache_file.exists():
            return {}
        
        try:
            with open(self.cache_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except Exception as e:
            logging.error(f"Failed to load translation cache: {str(e)}")
            return {}
    
    def _save_cache(self):
        """Save translation cache to file"""
        try:
            with open(self.cache_file, 'w', encoding='utf-8') as f:
                json.dump(self.cache, f, indent=2, ensure_ascii=False)
        except Exception as e:
            logging.error(f"Failed to save translation cache: {str(e)}")
    
    def _get_hash(self, text: str) -> str:
        """Get hash of text for cache key"""
        return hashlib.md5(text.encode('utf-8')).hexdigest()
    
    def get_translation(self, text: str) -> Optional[str]:
        """Get cached translation"""
        if not text or len(text.strip()) == 0:
            return ""
        
        text_hash = self._get_hash(text)
        cached = self.cache.get(text_hash)
        
        # ✅ Handle both old format (string) and new format (dict)
        if cached:
            if isinstance(cached, dict):
                return cached['translation']
            else:
                return cached  # Old format compatibility
        
        return None
    
    def save_translation(self, text: str, translation: str):
        """Save translation to cache with metadata"""
        if not text or len(text.strip()) == 0:
            return
        
        text_hash = self._get_hash(text)
        
        # ✅ NEW: Save with metadata
        self.cache[text_hash] = {
            'translation': translation,
            'source_lang': 'id',
            'target_lang': 'en',
            'translator': 'deep-translator (GoogleTranslator)',
            'timestamp': datetime.now().isoformat(),
            'text_length': len(text),
            'pipeline_version': VERSION
        }
        self._save_cache()
    
    def get_cache_size(self) -> int:
        """Get number of cached translations"""
        return len(self.cache)


# ==================== DECISION LOG MANAGER ====================
class DecisionLogManager:
    """Log processing decisions for reproducibility"""
    
    def __init__(self, log_dir: Path):
        self.log_dir = log_dir
        self.log_dir.mkdir(parents=True, exist_ok=True)
        self.decisions_file = self.log_dir / 'decision_log.jsonl'
    
    def log_decision(self, filename: str, decision_type: str, details: Dict):
        """Log a processing decision"""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'filename': filename,
            'decision_type': decision_type,
            'details': details
        }
        
        try:
            with open(self.decisions_file, 'a', encoding='utf-8') as f:
                f.write(json.dumps(entry, ensure_ascii=False) + '\n')
        except Exception as e:
            logging.error(f"Failed to log decision: {str(e)}")
    
    def log_reextraction(self, filename: str, reason: str, audio_dbfs: float, 
                        params: Dict, success: bool):
        """Log re-extraction decision"""
        self.log_decision(filename, 'reextraction', {
            'reason': reason,
            'audio_dbfs': audio_dbfs,
            'enhancement_params': params,
            'success': success
        })
    
    def log_transcript_source(self, filename: str, source: str, from_cache: bool = False):
        """Log transcript source"""
        self.log_decision(filename, 'transcript_source', {
            'source': source,
            'from_cache': from_cache
        })
    
    def log_translation(self, filename: str, from_cache: bool, text_length: int):
        """Log translation decision"""
        self.log_decision(filename, 'translation', {
            'from_cache': from_cache,
            'text_length': text_length
        })


# ==================== CHECKPOINT MANAGER ====================
class CheckpointManager:
    """Manage checkpoints for resume functionality"""
    
    def __init__(self, checkpoint_dir: Path):
        self.checkpoint_dir = checkpoint_dir
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.checkpoint_file = self.checkpoint_dir / 'processing_checkpoint.pkl'
        self.progress_file = self.checkpoint_dir / 'progress.json'
    
    def save_checkpoint(self, data: Dict):
        """Save checkpoint data"""
        try:
            with open(self.checkpoint_file, 'wb') as f:
                pickle.dump(data, f)
            
            # Save human-readable progress
            progress = {
                'timestamp': datetime.now().isoformat(),
                'processed_files': data.get('processed_files', []),
                'total_processed': len(data.get('processed_files', [])),
                'stats': data.get('stats', {})
            }
            with open(self.progress_file, 'w') as f:
                json.dump(progress, f, indent=2)
            
            return True
        except Exception as e:
            logging.error(f"Failed to save checkpoint: {str(e)}")
            return False
    
    def load_checkpoint(self) -> Optional[Dict]:
        """Load checkpoint data"""
        if not self.checkpoint_file.exists():
            return None
        
        try:
            with open(self.checkpoint_file, 'rb') as f:
                data = pickle.load(f)
            logging.info(f"✓ Checkpoint loaded: {len(data.get('processed_files', []))} files already processed")
            return data
        except Exception as e:
            logging.error(f"Failed to load checkpoint: {str(e)}")
            return None
    
    def clear_checkpoint(self):
        """Clear checkpoint files"""
        try:
            if self.checkpoint_file.exists():
                self.checkpoint_file.unlink()
            if self.progress_file.exists():
                self.progress_file.unlink()
            logging.info("✓ Checkpoint cleared")
        except Exception as e:
            logging.error(f"Failed to clear checkpoint: {str(e)}")
    
    def checkpoint_exists(self) -> bool:
        """Check if checkpoint exists"""
        return self.checkpoint_file.exists()


# ==================== INDONESIAN NUMBER CONVERTER ====================
class IndonesianNumberConverter:
    """Convert numbers to Indonesian words (0 - Trillion)"""
    
    def __init__(self):
        self.ones = ['', 'satu', 'dua', 'tiga', 'empat', 'lima', 'enam', 'tujuh', 'delapan', 'sembilan']
        self.teens = ['sepuluh', 'sebelas', 'dua belas', 'tiga belas', 'empat belas', 
                      'lima belas', 'enam belas', 'tujuh belas', 'delapan belas', 'sembilan belas']
        self.tens = ['', '', 'dua puluh', 'tiga puluh', 'empat puluh', 'lima puluh', 
                     'enam puluh', 'tujuh puluh', 'delapan puluh', 'sembilan puluh']
    
    def convert_below_hundred(self, num):
        if num == 0:
            return 'nol'
        elif num < 10:
            return self.ones[num]
        elif num < 20:
            return self.teens[num - 10]
        else:
            tens_digit = num // 10
            ones_digit = num % 10
            if ones_digit == 0:
                return self.tens[tens_digit]
            else:
                return f"{self.tens[tens_digit]} {self.ones[ones_digit]}"
    
    def convert_below_thousand(self, num):
        if num < 100:
            return self.convert_below_hundred(num)
        
        hundreds_digit = num // 100
        remainder = num % 100
        
        if hundreds_digit == 1:
            result = 'seratus'
        else:
            result = f"{self.ones[hundreds_digit]} ratus"
        
        if remainder > 0:
            result += f" {self.convert_below_hundred(remainder)}"
        
        return result
    
    def convert_number(self, num):
        if num == 0:
            return 'nol'
        
        if num < 0:
            return f"minus {self.convert_number(abs(num))}"
        
        if num >= 1000000000000:
            trillions = num // 1000000000000
            remainder = num % 1000000000000
            result = 'satu triliun' if trillions == 1 else f"{self.convert_below_thousand(trillions)} triliun"
            if remainder > 0:
                result += f" {self.convert_number(remainder)}"
            return result
        
        if num >= 1000000000:
            billions = num // 1000000000
            remainder = num % 1000000000
            result = 'satu miliar' if billions == 1 else f"{self.convert_below_thousand(billions)} miliar"
            if remainder > 0:
                result += f" {self.convert_number(remainder)}"
            return result
        
        if num >= 1000000:
            millions = num // 1000000
            remainder = num % 1000000
            result = 'satu juta' if millions == 1 else f"{self.convert_below_thousand(millions)} juta"
            if remainder > 0:
                result += f" {self.convert_number(remainder)}"
            return result
        
        if num >= 1000:
            thousands = num // 1000
            remainder = num % 1000
            result = 'seribu' if thousands == 1 else f"{self.convert_below_thousand(thousands)} ribu"
            if remainder > 0:
                result += f" {self.convert_below_thousand(remainder)}"
            return result
        
        return self.convert_below_thousand(num)
    
    def convert_decimal(self, num_str):
        parts = num_str.split('.')
        
        if len(parts) == 1:
            return self.convert_number(int(parts[0]))
        
        integer_part = int(parts[0]) if parts[0] else 0
        decimal_part = parts[1] if len(parts) > 1 and parts[1] else ''
        
        result = self.convert_number(integer_part)
        
        if decimal_part:
            result += ' koma'
            for digit in decimal_part:
                if digit.isdigit():
                    result += f" {self.ones[int(digit)]}"
        
        return result


def normalize_numbers_in_text(text):
    """Normalize all numbers in text to Indonesian words"""
    if not text or len(text.strip()) == 0:
        return text
    
    converter = IndonesianNumberConverter()
    number_pattern = r'\b\d+(?:[.,]\d+)*\b'
    
    def replace_number(match):
        num_str = match.group(0)
        num_str_clean = num_str.replace('.', '').replace(',', '.')
        
        try:
            if '.' in num_str_clean:
                return converter.convert_decimal(num_str_clean)
            else:
                num = int(num_str_clean)
                return converter.convert_number(num)
        except Exception as e:
            logging.warning(f"Failed to convert number '{num_str}': {str(e)}")
            return num_str
    
    normalized_text = re.sub(number_pattern, replace_number, text)
    normalized_text = re.sub(r'\s+', ' ', normalized_text).strip()
    
    return normalized_text


def extract_number_features(text):
    """Extract deception-specific number features"""
    if not text or len(text.strip()) == 0:
        return {
            'has_numbers': False, 'number_count': 0, 'has_vague_quantifiers': False,
            'has_exact_quantifiers': False, 'number_word_ratio': 0.0,
            'has_large_numbers': False, 'has_decimal_numbers': False
        }
    
    digit_pattern = r'\b\d+(?:[.,]\d+)*\b'
    numbers = re.findall(digit_pattern, text)
    
    number_words = r'\b(nol|satu|dua|tiga|empat|lima|enam|tujuh|delapan|sembilan|sepuluh|sebelas|belas|puluh|ratus|ribu|juta|miliar|triliun)\b'
    number_word_matches = re.findall(number_words, text.lower())
    
    vague_pattern = r'\b(sekitar|kira-kira|kurang lebih|hampir|lebih dari|kurang dari|lebih kurang|sekitaran|kisaran)\b'
    has_vague = bool(re.search(vague_pattern, text.lower()))
    
    exact_pattern = r'\b(tepat|persis|tepatnya|pastinya|pasti|exactly)\b'
    has_exact = bool(re.search(exact_pattern, text.lower()))
    
    large_numbers = []
    for n in numbers:
        try:
            val = float(n.replace('.', '').replace(',', '.'))
            if val > 1000:
                large_numbers.append(n)
        except ValueError:
            pass
    
    decimal_numbers = [n for n in numbers if '.' in n or ',' in n]
    
    total_words = len(text.split())
    number_word_ratio = len(number_word_matches) / total_words if total_words > 0 else 0.0
    
    return {
        'has_numbers': len(numbers) > 0 or len(number_word_matches) > 0,
        'number_count': len(numbers) + len(number_word_matches),
        'has_vague_quantifiers': has_vague,
        'has_exact_quantifiers': has_exact,
        'number_word_ratio': number_word_ratio,
        'has_large_numbers': len(large_numbers) > 0,
        'has_decimal_numbers': len(decimal_numbers) > 0
    }


# ==================== CONFIGURATION ====================
class I3DConfig:
    """Configuration for I3D Dataset - Scientific Data Ready"""
    
    DEFAULT_CONFIG = {
        'dataset': {
            'name': 'I3D',
            'version': '1.0',
            'language': 'Indonesian',
            'description': 'Indonesian Interrogation Deception Dataset',
            'citation': 'Please cite: [Your Paper]'
        },
        'audio': {
            'sampling_rate': 16000,
            'librosa_params': {
                'n_fft': 2048,
                'hop_length': 512,
                'win_length': 2048,
                'window': 'hann',
                'center': True,
                'pad_mode': 'constant'
            },
            'enhancement': {
                'extreme_boost_threshold': -55,
                'max_boost': 40,
                'target_dbfs': -18,
                'silence_threshold': -50,
                'min_pause_duration': 200,
                'compression_ratio': 6.0,
                'short_speech_threshold': 2000,
                'fixed_params': True
            }
        },
        'video': {
            'sample_rate': 1,
            'zoom_levels': [1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0],
            'preprocessing': {
                'clahe_clip_limit': 4.0,
                'clahe_tile_size': (4, 4),
                'gamma_correction': 1.4,
                'unsharp_strength': 1.8
            }
        },
        'mediapipe': {
            'face_mesh': {
                'static_image_mode': False,
                'max_num_faces': 1,
                'refine_landmarks': True,
                'min_detection_confidence': 0.2,
                'min_tracking_confidence': 0.2
            },
            'pose': {
                'static_image_mode': False,
                'model_complexity': 1,
                'min_detection_confidence': 0.3,
                'min_tracking_confidence': 0.3
            }
        },
        'speech_recognition': {
            'language': 'id-ID',
            'max_attempts': 7,
            'deterministic_mode': True,  # ✅ CHANGED: True by default for reproducibility
            'configs': [
                {'energy_threshold': 300, 'pause_threshold': 0.8, 'phrase_threshold': 0.3},
                {'energy_threshold': 150, 'pause_threshold': 0.6, 'phrase_threshold': 0.2},
                {'energy_threshold': 100, 'pause_threshold': 0.5, 'phrase_threshold': 0.1},
                {'energy_threshold': 50, 'pause_threshold': 0.4, 'phrase_threshold': 0.1},
                {'energy_threshold': 30, 'pause_threshold': 0.3, 'phrase_threshold': 0.05},
                {'energy_threshold': 20, 'pause_threshold': 0.2, 'phrase_threshold': 0.05},
                {'energy_threshold': 10, 'pause_threshold': 0.1, 'phrase_threshold': 0.01}
            ]
        },
        'text': {
            'normalize_numbers': True,
            'extract_number_features': True,
            'translate_to_english': True,
            'use_manual_transcripts': True,
            'use_transcript_cache': True,
            'use_translation_cache': True,
            'force_retranscribe': False,
            'sentiment_analysis_note': 'TextBlob sentiment for Indonesian is proxy only'
        },
        'reextraction': {
            'enabled': True,
            'auto_trigger': True,
            'log_decisions': True
        },
        'checkpoint': {
            'enabled': True,
            'save_interval': 5
        },
        'output': {
            'generate_figures': True,
            'generate_markdown_report': True,
            'generate_data_dictionary': True,
            'generate_run_manifest': True,
            'figure_dpi': 300,
            'csv_encoding': 'utf-8'
        }
    }
    
    def __init__(self, config_path: Optional[str] = None):
        self.config = self._deep_copy(self.DEFAULT_CONFIG)
        
        if config_path and os.path.exists(config_path):
            self.load_from_file(config_path)
    
    def _deep_copy(self, d):
        """Deep copy dictionary"""
        import copy
        return copy.deepcopy(d)
    
    def load_from_file(self, config_path: str):
        try:
            with open(config_path, 'r') as f:
                user_config = yaml.safe_load(f)
                self._merge_config(self.config, user_config)
            logging.info(f"✓ Configuration loaded from: {config_path}")
        except Exception as e:
            logging.warning(f"Failed to load config: {str(e)}. Using defaults.")
    
    def _merge_config(self, base: dict, update: dict):
        for key, value in update.items():
            if key in base and isinstance(base[key], dict) and isinstance(value, dict):
                self._merge_config(base[key], value)
            else:
                base[key] = value
    
    def save_to_file(self, config_path: str):
        with open(config_path, 'w') as f:
            yaml.dump(self.config, f, default_flow_style=False)
        logging.info(f"✓ Configuration saved to: {config_path}")
    
    def get(self, key_path: str, default=None):
        keys = key_path.split('.')
        value = self.config
        for key in keys:
            if isinstance(value, dict) and key in value:
                value = value[key]
            else:
                return default
        return value


# ==================== PATH MANAGER ====================
class I3DPathManager:
    """Path management for I3D Dataset"""
    
    def __init__(self, base_dir: str):
        self.base_dir = Path(base_dir)
        self.dataset_name = 'I3D'
        
        dataset_base = self.base_dir / "dataset"
        
        self.paths = {
            # Source (INPUT)
            'source_raw': dataset_base / "raw" / "I3D",
            'source_lie': dataset_base / "raw" / "I3D" / "lie",
            'source_truth': dataset_base / "raw" / "I3D" / "truth",
            
            # Processed (OUTPUT)
            'processed': dataset_base / "processed" / "I3D",
            'audio': dataset_base / "processed" / "I3D" / "audio",
            'audio_wav': dataset_base / "processed" / "I3D" / "audio" / "wav",
            'audio_wav_lie': dataset_base / "processed" / "I3D" / "audio" / "wav" / "lie",
            'audio_wav_truth': dataset_base / "processed" / "I3D" / "audio" / "wav" / "truth",
            'audio_enhanced': dataset_base / "processed" / "I3D" / "audio" / "enhanced",
            'audio_enhanced_lie': dataset_base / "processed" / "I3D" / "audio" / "enhanced" / "lie",
            'audio_enhanced_truth': dataset_base / "processed" / "I3D" / "audio" / "enhanced" / "truth",
            'text': dataset_base / "processed" / "I3D" / "text",
            'visual': dataset_base / "processed" / "I3D" / "visual",
            'multimodal': dataset_base / "processed" / "I3D" / "multimodal",
            'metadata': dataset_base / "metadata" / "I3D",
            'validation': dataset_base / "validation" / "I3D",
            'quality_reports': dataset_base / "validation" / "I3D" / "quality_reports",
            'statistical': dataset_base / "validation" / "I3D" / "statistical_analyses",
            'figures': dataset_base / "figures" / "I3D",
            'exploratory_figs': dataset_base / "figures" / "I3D" / "exploratory",
            'logs': dataset_base / "_logs" / "I3D",
            'reextraction': dataset_base / "processed" / "I3D" / "reextraction",
            'checkpoints': dataset_base / "_checkpoints" / "I3D",
            'cache': dataset_base / "_cache" / "I3D",
            'decision_logs': dataset_base / "_logs" / "I3D" / "decisions"
        }
    
    def create_directories(self):
        for path in self.paths.values():
            path.mkdir(parents=True, exist_ok=True)
        print(f"✓ Directory structure created for I3D")
    
    def get_log_path(self) -> Tuple[Path, str]:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        log_path = self.paths['logs'] / f"extraction_{timestamp}.log"
        return log_path, timestamp


# ==================== LOGGER ====================
class I3DLogger:
    """Logger for I3D Dataset"""
    
    def __init__(self, log_path: Path):
        self.log_path = log_path
        self.start_time = datetime.now()
        
        for handler in logging.root.handlers[:]:
            logging.root.removeHandler(handler)
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - [I3D] - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(log_path, encoding='utf-8'),
                logging.StreamHandler()
            ]
        )
    
    def log(self, message: str):
        logging.info(message)
    
    def log_error(self, message: str):
        logging.error(message)
    
    def log_warning(self, message: str):
        logging.warning(message)
    
    def finalize(self):
        end_time = datetime.now()
        duration = end_time - self.start_time
        logging.info("\n" + "="*70)
        logging.info(f"Finished at: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        logging.info(f"Total duration: {duration}")


# ==================== VIDEO PREPROCESSOR ====================
class VideoPreprocessor:
    """Video preprocessing for I3D"""
    
    def __init__(self, config: I3DConfig):
        self.config = config
        try:
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
        except Exception as e:
            logging.error(f"Failed to load Haar cascades: {str(e)}")
            self.face_cascade = None
            self.eye_cascade = None
    
    def zoom_frame(self, frame: np.ndarray, zoom_factor: float = 1.5) -> np.ndarray:
        height, width = frame.shape[:2]
        center_x, center_y = width // 2, height // 2
        
        crop_width = int(width / zoom_factor)
        crop_height = int(height / zoom_factor)
        
        x1 = max(0, center_x - crop_width // 2)
        y1 = max(0, center_y - crop_height // 2)
        x2 = min(width, x1 + crop_width)
        y2 = min(height, y1 + crop_height)
        
        cropped = frame[y1:y2, x1:x2]
        zoomed = cv2.resize(cropped, (width, height), interpolation=cv2.INTER_CUBIC)
        
        return zoomed
    
    def enhance_eye_region_for_iris(self, frame: np.ndarray) -> Tuple[np.ndarray, Optional[List]]:
        if self.face_cascade is None or self.eye_cascade is None:
            return frame, None
            
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if len(faces) == 0:
            return frame, None
        
        result = frame.copy()
        eye_regions = []
        
        for (x, y, w, h) in faces:
            roi_y_start = y
            roi_y_end = y + int(h * 0.6)
            roi_gray = gray[roi_y_start:roi_y_end, x:x+w]
            roi_color = result[roi_y_start:roi_y_end, x:x+w]
            
            eyes = self.eye_cascade.detectMultiScale(roi_gray, 1.1, 5)
            
            for (ex, ey, ew, eh) in eyes:
                expansion = 0.3
                ex_exp = max(0, int(ex - ew * expansion))
                ey_exp = max(0, int(ey - eh * expansion))
                ew_exp = min(roi_color.shape[1] - ex_exp, int(ew * (1 + 2 * expansion)))
                eh_exp = min(roi_color.shape[0] - ey_exp, int(eh * (1 + 2 * expansion)))
                
                eye_roi = roi_color[ey_exp:ey_exp+eh_exp, ex_exp:ex_exp+ew_exp]
                
                if eye_roi.size == 0:
                    continue
                
                lab = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2LAB)
                l, a, b = cv2.split(lab)
                
                clip_limit = self.config.get('video.preprocessing.clahe_clip_limit', 4.0)
                tile_size = self.config.get('video.preprocessing.clahe_tile_size', (4, 4))
                
                clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
                l_enhanced = clahe.apply(l)
                lab_enhanced = cv2.merge([l_enhanced, a, b])
                eye_enhanced = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
                
                kernel = np.array([[-1, -1, -1], [-1, 10, -1], [-1, -1, -1]])
                eye_enhanced = cv2.filter2D(eye_enhanced, -1, kernel)
                eye_enhanced = cv2.convertScaleAbs(eye_enhanced, alpha=1.2, beta=15)
                
                roi_color[ey_exp:ey_exp+eh_exp, ex_exp:ex_exp+ew_exp] = eye_enhanced
                eye_regions.append((x + ex_exp, roi_y_start + ey_exp, ew_exp, eh_exp))
        
        return result, eye_regions
    
    def preprocess_pipeline_ultra(self, frame: np.ndarray) -> np.ndarray:
        # Step 1: Denoise
        frame = cv2.bilateralFilter(frame, d=9, sigmaColor=75, sigmaSpace=75)
        
        # Step 2: Adaptive brightness
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        mean_brightness = np.mean(gray)
        
        if mean_brightness < 50:
            alpha, beta = 2.5, 80
        elif mean_brightness < 100:
            alpha, beta = 1.8, 50
        elif mean_brightness < 120:
            alpha, beta = 1.3, 20
        elif mean_brightness > 180:
            alpha, beta = 0.8, -20
        else:
            alpha, beta = 1.0, 0
        
        frame = cv2.convertScaleAbs(frame, alpha=alpha, beta=beta)
        
        # Step 3: Enhance eye region for iris
        frame, eye_regions = self.enhance_eye_region_for_iris(frame)
        
        # Step 4: Gamma correction
        gamma = self.config.get('video.preprocessing.gamma_correction', 1.4)
        inv_gamma = 1.0 / gamma
        table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
        frame = cv2.LUT(frame, table)
        
        # Step 5: Sharpen
        kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
        frame = cv2.filter2D(frame, -1, kernel)
        
        # Step 6: Unsharp mask
        strength = self.config.get('video.preprocessing.unsharp_strength', 1.8)
        blurred = cv2.GaussianBlur(frame, (0, 0), 1.5)
        frame = cv2.addWeighted(frame, 1.0 + strength, blurred, -strength, 0)
        
        # Step 7: Final CLAHE
        lab = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        l_enhanced = clahe.apply(l)
        lab_enhanced = cv2.merge([l_enhanced, a, b])
        frame = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
        
        return frame


# ==================== AUDIO PROCESSOR ====================
class AudioProcessor:
    """Audio processing for I3D with deterministic enhancement"""
    
    def __init__(self, config: I3DConfig):
        self.config = config
    
    def get_enhancement_params(self, audio_dbfs: float, ultra_mode: bool = False) -> Dict:
        """Get fixed enhancement parameters based on audio level"""
        params = {
            'ultra_mode': ultra_mode,
            'original_dbfs': audio_dbfs
        }
        
        # Fixed thresholds (not adaptive)
        extreme_threshold = self.config.get('audio.enhancement.extreme_boost_threshold', -55)
        
        if ultra_mode or audio_dbfs < -45:
            if audio_dbfs < extreme_threshold:
                params['boost_amount'] = 40  # Fixed max boost
                params['boost_type'] = 'ultra'
            elif audio_dbfs < -45:
                params['boost_amount'] = 35  # Fixed aggressive boost
                params['boost_type'] = 'aggressive'
            else:
                params['boost_amount'] = 20
                params['boost_type'] = 'moderate'
        elif audio_dbfs < -35:
            params['boost_amount'] = 20
            params['boost_type'] = 'moderate'
        else:
            params['boost_amount'] = 0
            params['boost_type'] = 'none'
        
        # Fixed target
        params['target_dbfs'] = -18
        params['silence_threshold'] = -50
        params['compression_ratio'] = 6.0 if not ultra_mode else 10.0
        
        return params
    
    def enhance_audio(self, audio_path: Path, logger: I3DLogger, 
                     ultra_mode: bool = False) -> Tuple[Path, Dict]:
        """Enhanced audio with fixed parameters for reproducibility"""
        try:
            if not audio_path.exists():
                logger.log_error(f"Audio file not found: {audio_path}")
                return audio_path, {}
            
            if ultra_mode:
                enhanced_path = audio_path.parent / f"{audio_path.stem}_ultra_enhanced.wav"
            else:
                enhanced_path = audio_path.parent / f"{audio_path.stem}_enhanced.wav"
            
            if enhanced_path.exists():
                logger.log(f"Enhanced audio exists: {enhanced_path.name}")
                audio = AudioSegment.from_wav(audio_path)
                params = self.get_enhancement_params(audio.dBFS, ultra_mode)
                return enhanced_path, params
            
            audio = AudioSegment.from_wav(audio_path)
            original_duration = len(audio)
            original_dbfs = audio.dBFS
            
            # Get fixed parameters
            params = self.get_enhancement_params(original_dbfs, ultra_mode)
            
            logger.log(f"  └─ 📊 Original: {original_dbfs:.1f} dBFS, {original_duration/1000:.1f}s")
            logger.log(f"  └─ 🔧 Enhancement params: {params['boost_type']}, boost={params['boost_amount']}dB")
            
            enhancement_applied = []
            
            # Apply fixed boost
            if params['boost_amount'] > 0:
                audio = audio + params['boost_amount']
                enhancement_applied.append(f"{params['boost_type']} boost: +{params['boost_amount']}dB")
            
            # Normalize
            normalized_audio = audio.normalize(headroom=0.05)
            enhancement_applied.append("Normalized")
            
            # Target boost (fixed)
            if normalized_audio.dBFS < params['target_dbfs']:
                boost_amount = min(abs(normalized_audio.dBFS - params['target_dbfs']), 15)
                normalized_audio = normalized_audio + boost_amount
                enhancement_applied.append(f"Final boost: +{boost_amount:.1f}dB")
            
            # Trim silence (fixed threshold)
            silence_threshold = params['silence_threshold']
            chunk_length_ms = 10
            padding_ms = 100
            
            start_trim = 0
            for i in range(0, len(normalized_audio), chunk_length_ms):
                chunk = normalized_audio[i:i+chunk_length_ms]
                if len(chunk) > 0 and chunk.dBFS > silence_threshold:
                    start_trim = max(0, i - padding_ms)
                    break
            
            end_trim = len(normalized_audio)
            for i in range(len(normalized_audio), 0, -chunk_length_ms):
                chunk = normalized_audio[i-chunk_length_ms:i]
                if len(chunk) > 0 and chunk.dBFS > silence_threshold:
                    end_trim = min(len(normalized_audio), i + padding_ms)
                    break
            
            if start_trim > 0 or end_trim < len(normalized_audio):
                trimmed_audio = normalized_audio[start_trim:end_trim]
                trimmed_ms = (start_trim + (len(normalized_audio) - end_trim))
                if trimmed_ms > 50:
                    enhancement_applied.append(f"Trimmed: {trimmed_ms}ms")
                normalized_audio = trimmed_audio
            
            # Filtering (fixed)
            filtered_audio = normalized_audio.high_pass_filter(80).low_pass_filter(3000)
            enhancement_applied.append("Filter (80-3000Hz)")
            
            # Compression (fixed ratio)
            compressed_audio = filtered_audio.compress_dynamic_range(
                threshold=-15.0, 
                ratio=params['compression_ratio'],
                attack=3.0, 
                release=40.0
            )
            enhancement_applied.append(f"Compression (ratio={params['compression_ratio']})")
            
            # De-clipping
            samples = np.array(compressed_audio.get_array_of_samples())
            max_val = np.max(np.abs(samples))
            clipping_threshold = 32767 * 0.99
            
            if max_val > clipping_threshold:
                samples_normalized = samples / 32768.0
                samples_fixed = np.tanh(samples_normalized * 0.8) * 32767
                compressed_audio = compressed_audio._spawn(samples_fixed.astype(np.int16).tobytes())
                enhancement_applied.append("De-clipped")
            
            # Final normalize
            final_audio = compressed_audio.normalize(headroom=0.1)
            
            # Export
            final_audio.export(str(enhanced_path), format="wav")
            
            final_duration = len(final_audio)
            improvement = final_audio.dBFS - original_dbfs
            
            logger.log(f"✓ Enhanced: {audio_path.name}")
            logger.log(f"  └─ 📈 Improvement: {original_dbfs:.1f} → {final_audio.dBFS:.1f} dBFS ({improvement:+.1f}dB)")
            logger.log(f"  └─ ⏱️ Duration: {original_duration/1000:.1f}s → {final_duration/1000:.1f}s")
            logger.log(f"  └─ 🔧 Applied: {', '.join(enhancement_applied)}")
            
            params['final_dbfs'] = final_audio.dBFS
            params['enhancement_applied'] = enhancement_applied
            
            return enhanced_path, params
            
        except Exception as e:
            logger.log_error(f"Failed to enhance audio: {str(e)}")
            import traceback
            logger.log_error(traceback.format_exc())
            return audio_path, {}
    
    def extract_text(self, audio_path: Path, logger: I3DLogger, 
                     ultra_mode: bool = False, filename: str = None,
                     transcript_cache: TranscriptCacheManager = None,
                     decision_log: DecisionLogManager = None,
                     force_retranscribe: bool = False) -> Tuple[Optional[str], bool, Dict]:
        """Extract text with caching and decision logging"""
        
        metadata = {
            'source': 'unknown',
            'from_cache': False,
            'attempt': 0,
            'mode': 'normal' if not ultra_mode else 'ultra',
            'stt_energy_threshold': 0.0
        }
        
        # Check force retranscribe
        if not force_retranscribe:
            # Check transcript cache first
            if transcript_cache and self.config.get('text.use_transcript_cache', True):
                cached = transcript_cache.load_transcript(filename)
                if cached:
                    logger.log(f"✓ Using CACHED TRANSCRIPT: '{cached['text']}'")
                    logger.log(f"  └─ Source: {cached['source']}, Mode: {cached['mode']}")
                    
                    if decision_log:
                        decision_log.log_transcript_source(filename, cached['source'], from_cache=True)
                    
                    metadata['source'] = cached['source']
                    metadata['from_cache'] = True
                    metadata['attempt'] = cached.get('attempt', 0)
                    metadata['mode'] = cached.get('mode', 'normal')
                    metadata['stt_energy_threshold'] = cached.get('stt_energy_threshold', 0.0)
                    
                    return cached['text'], True, metadata
            
            # Check manual transcript
            if filename and self.config.get('text.use_manual_transcripts', True):
                if filename in MANUAL_TRANSCRIPTS:
                    manual_text = MANUAL_TRANSCRIPTS[filename]
                    logger.log(f"✓ Using MANUAL TRANSCRIPT: '{manual_text}'")
                    
                    # Save to cache
                    if transcript_cache:
                        transcript_cache.save_transcript(
                            filename, manual_text, 'manual', 
                            attempt=0, mode='manual'
                        )
                    
                    if decision_log:
                        decision_log.log_transcript_source(filename, 'manual', from_cache=False)
                    
                    metadata['source'] = 'manual'
                    metadata['from_cache'] = False
                    
                    return manual_text, True, metadata
        else:
            logger.log(f"⚠️ FORCE RETRANSCRIBE enabled - ignoring cache")
        
        # Proceed with automatic speech recognition
        recognizer = sr.Recognizer()
        
        # Deterministic mode
        deterministic_mode = self.config.get('speech_recognition.deterministic_mode', False)
        if deterministic_mode:
            recognizer.dynamic_energy_threshold = False
            logger.log(f"  └─ 🔒 Deterministic mode: dynamic_energy_threshold=False")
        else:
            recognizer.dynamic_energy_threshold = True
        
        configs = self.config.get('speech_recognition.configs', [])
        max_attempts = len(configs) if ultra_mode else 3
        language = self.config.get('speech_recognition.language', 'id-ID')
        
        for attempt in range(max_attempts):
            try:
                config = configs[attempt]
                
                recognizer.energy_threshold = config['energy_threshold']
                recognizer.pause_threshold = config['pause_threshold']
                recognizer.phrase_threshold = config['phrase_threshold']
                recognizer.non_speaking_duration = config.get('non_speaking_duration', 0.3)
                
                with sr.AudioFile(str(audio_path)) as source:
                    if not deterministic_mode:
                        recognizer.adjust_for_ambient_noise(source, duration=0.2)
                    audio_data = recognizer.record(source)
                
                # Log actual energy threshold used
                actual_threshold = recognizer.energy_threshold
                logger.log(f"  └─ Attempt {attempt+1}: energy_threshold={actual_threshold:.1f}")
                
                text = recognizer.recognize_google(audio_data, language=language)
                
                if text and len(text.strip()) > 0:
                    logger.log(f"✓ Speech recognition success (attempt {attempt+1}): '{text}'")
                    
                    # Get audio dBFS for metadata
                    try:
                        audio = AudioSegment.from_wav(audio_path)
                        audio_dbfs = audio.dBFS
                    except:
                        audio_dbfs = 0.0
                    
                    # Save to cache
                    if transcript_cache:
                        transcript_cache.save_transcript(
                            filename, text, 
                            'stt_ultra' if ultra_mode else 'stt',
                            attempt=attempt+1,
                            mode='ultra' if ultra_mode else 'normal',
                            audio_dbfs=audio_dbfs,
                            stt_energy_threshold=actual_threshold
                        )
                    
                    if decision_log:
                        decision_log.log_transcript_source(
                            filename, 
                            'stt_ultra' if ultra_mode else 'stt',
                            from_cache=False
                        )
                    
                    metadata['source'] = 'stt_ultra' if ultra_mode else 'stt'
                    metadata['from_cache'] = False
                    metadata['attempt'] = attempt + 1
                    metadata['stt_energy_threshold'] = actual_threshold
                    
                    return text, True, metadata
            
            except sr.UnknownValueError:
                if attempt < max_attempts - 1:
                    logger.log(f"  └─ Attempt {attempt+1} failed, trying next...")
                    continue
                else:
                    logger.log_warning(f"  └─ All {max_attempts} attempts failed")
            
            except sr.RequestError as e:
                logger.log_error(f"  └─ Google API error: {str(e)}")
                break
            
            except Exception as e:
                logger.log_error(f"  └─ Attempt {attempt+1} error: {str(e)}")
                continue
        
        metadata['source'] = 'failed'
        return None, False, metadata


# ==================== LANDMARK EXTRACTOR ====================
class LandmarkExtractor:
    """Landmark extraction for I3D"""
    
    def __init__(self, config: I3DConfig, preprocessor: VideoPreprocessor):
        self.config = config
        self.preprocessor = preprocessor
    
    def extract_with_multi_strategy(self, frame: np.ndarray, face_mesh, 
                                    iris_focus: bool = False) -> Tuple[Optional[Dict], int, Optional[str]]:
        best_result = None
        best_iris_count = 0
        best_strategy = None
        
        zoom_levels = self.config.get('video.zoom_levels', [1.5, 2.0, 2.5, 3.0, 4.0, 5.0, 6.0, 8.0, 10.0])
        
        # Strategy 1: Ultra preprocessing + multi-level zoom
        for zoom in zoom_levels:
            try:
                if zoom == 1.0:
                    processed_frame = frame
                else:
                    processed_frame = self.preprocessor.zoom_frame(frame.copy(), zoom)
                
                processed_frame = self.preprocessor.preprocess_pipeline_ultra(processed_frame)
                frame_rgb = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(frame_rgb)
                
                if results.multi_face_landmarks:
                    landmarks_dict = {}
                    for face_landmarks in results.multi_face_landmarks:
                        for idx, landmark in enumerate(face_landmarks.landmark):
                            landmarks_dict[idx] = [landmark.x, landmark.y, landmark.z]
                    
                    iris_count = sum(1 for i in range(468, 478) if i in landmarks_dict)
                    
                    if iris_count > best_iris_count:
                        best_result = landmarks_dict
                        best_iris_count = iris_count
                        best_strategy = f"zoom_{zoom}x_ultra"
                    
                    if iris_count >= 9:
                        return landmarks_dict, iris_count, best_strategy
            
            except Exception as e:
                continue
        
        # Strategy 2: Extreme brightness + zoom
        if iris_focus and best_iris_count < 8:
            for zoom in [3.0, 5.0, 8.0]:
                try:
                    zoomed = self.preprocessor.zoom_frame(frame.copy(), zoom)
                    extreme_bright = cv2.convertScaleAbs(zoomed, alpha=2.5, beta=80)
                    processed = self.preprocessor.preprocess_pipeline_ultra(extreme_bright)
                    
                    frame_rgb = cv2.cvtColor(processed, cv2.COLOR_BGR2RGB)
                    results = face_mesh.process(frame_rgb)
                    
                    if results.multi_face_landmarks:
                        landmarks_dict = {}
                        for face_landmarks in results.multi_face_landmarks:
                            for idx, landmark in enumerate(face_landmarks.landmark):
                                landmarks_dict[idx] = [landmark.x, landmark.y, landmark.z]
                        
                        iris_count = sum(1 for i in range(468, 478) if i in landmarks_dict)
                        
                        if iris_count > best_iris_count:
                            best_result = landmarks_dict
                            best_iris_count = iris_count
                            best_strategy = f"zoom_{zoom}x_extreme"
                        
                        if iris_count >= 9:
                            return landmarks_dict, iris_count, best_strategy
                
                except Exception as e:
                    continue
        
        return best_result, best_iris_count, best_strategy


# ==================== PAUSE ANALYZER ====================
def extract_pause_silence_features(audio_path: Path, config: I3DConfig, logger: I3DLogger) -> Dict:
    """Extract pause and silence features"""
    try:
        y, sr = librosa.load(str(audio_path), sr=16000)
        duration = len(y) / sr
        
        audio = AudioSegment.from_wav(str(audio_path))
        
        silence_threshold = config.get('audio.enhancement.silence_threshold', -50)
        min_pause_duration = config.get('audio.enhancement.min_pause_duration', 200)
        chunk_length_ms = 50
        
        pauses = []
        current_pause_start = None
        current_pause_duration = 0
        
        for i in range(0, len(audio), chunk_length_ms):
            chunk = audio[i:i+chunk_length_ms]
            
            if len(chunk) == 0:
                continue
            
            is_silent = chunk.dBFS < silence_threshold
            
            if is_silent:
                if current_pause_start is None:
                    current_pause_start = i
                current_pause_duration += chunk_length_ms
            else:
                if current_pause_start is not None and current_pause_duration >= min_pause_duration:
                    pauses.append({
                        'start': current_pause_start,
                        'duration': current_pause_duration,
                        'end': current_pause_start + current_pause_duration
                    })
                
                current_pause_start = None
                current_pause_duration = 0
        
        if current_pause_start is not None and current_pause_duration >= min_pause_duration:
            pauses.append({
                'start': current_pause_start,
                'duration': current_pause_duration,
                'end': current_pause_start + current_pause_duration
            })
        
        num_pauses = len(pauses)
        total_pause_duration = sum(p['duration'] for p in pauses) / 1000
        
        if num_pauses > 0:
            avg_pause_duration = total_pause_duration / num_pauses
            longest_pause = max(p['duration'] for p in pauses) / 1000
            pause_frequency = num_pauses / duration if duration > 0 else 0
            
            early_pauses = sum(1 for p in pauses if p['start'] < len(audio) * 0.33)
            middle_pauses = sum(1 for p in pauses if len(audio) * 0.33 <= p['start'] < len(audio) * 0.67)
            late_pauses = sum(1 for p in pauses if p['start'] >= len(audio) * 0.67)
        else:
            avg_pause_duration = 0
            longest_pause = 0
            pause_frequency = 0
            early_pauses = 0
            middle_pauses = 0
            late_pauses = 0
        
        speech_duration = duration - total_pause_duration
        speech_rate = speech_duration / duration if duration > 0 else 0
        pause_ratio = total_pause_duration / duration if duration > 0 else 0
        
        hesitation_score = (
            0.3 * pause_ratio +
            0.3 * (pause_frequency / 2) +
            0.2 * min(longest_pause / 2, 1.0) +
            0.2 * min(num_pauses / 10, 1.0)
        )
        
        if num_pauses > 1:
            pause_durations = [p['duration'] / 1000 for p in pauses]
            pause_std = np.std(pause_durations)
            pause_variability = pause_std / avg_pause_duration if avg_pause_duration > 0 else 0
        else:
            pause_std = 0
            pause_variability = 0
        
        features = {
            'num_pauses': num_pauses,
            'total_pause_duration': float(total_pause_duration),
            'avg_pause_duration': float(avg_pause_duration),
            'longest_pause_duration': float(longest_pause),
            'pause_frequency': float(pause_frequency),
            'speech_duration': float(speech_duration),
            'speech_rate': float(speech_rate),
            'pause_ratio': float(pause_ratio),
            'early_pauses': int(early_pauses),
            'middle_pauses': int(middle_pauses),
            'late_pauses': int(late_pauses),
            'pause_std': float(pause_std),
            'pause_variability': float(pause_variability),
            'hesitation_score': float(hesitation_score),
            'has_long_pauses': longest_pause > 1.5,
            'has_frequent_pauses': pause_frequency > 1.0,
            'has_high_pause_ratio': pause_ratio > 0.3
        }
        
        logger.log(f"✓ Pause features extracted")
        logger.log(f"  └─ Pauses: {num_pauses}, Hesitation: {hesitation_score:.3f}")
        
        return features
        
    except Exception as e:
        logger.log_error(f"Failed to extract pause features: {str(e)}")
        return {
            'num_pauses': 0, 'total_pause_duration': 0, 'avg_pause_duration': 0,
            'longest_pause_duration': 0, 'pause_frequency': 0, 'speech_duration': 0,
            'speech_rate': 0, 'pause_ratio': 0, 'early_pauses': 0, 'middle_pauses': 0,
            'late_pauses': 0, 'pause_std': 0, 'pause_variability': 0, 'hesitation_score': 0,
            'has_long_pauses': False, 'has_frequent_pauses': False, 'has_high_pause_ratio': False
        }


# ==================== AUDIO FEATURES (FIXED TO 94) ====================
def extract_audio_features(audio_path: Path, config: I3DConfig, logger: I3DLogger) -> Optional[np.ndarray]:
    """
    Extract 94 audio features with explicit librosa parameters
    
    Features breakdown:
    - MFCC: 13 mean + 13 std = 26
    - Delta MFCC: 13 mean + 13 std = 26
    - Delta2 MFCC: 13 mean + 13 std = 26
    - Bark Energy: 2 (mean, std)
    - Delta Energy: 2 (mean, std)
    - Delta2 Energy: 2 (mean, std)
    - Spectral Centroid: 2 (mean, std)
    - Spectral Bandwidth: 2 (mean, std)
    - Spectral Rolloff: 2 (mean, std)
    - ZCR: 2 (mean, std)
    - Chroma: 2 (mean, std)
    Total: 94 features
    """
    try:
        # Get librosa parameters from config
        librosa_params = config.get('audio.librosa_params', {})
        n_fft = librosa_params.get('n_fft', 2048)
        hop_length = librosa_params.get('hop_length', 512)
        win_length = librosa_params.get('win_length', 2048)
        window = librosa_params.get('window', 'hann')
        center = librosa_params.get('center', True)
        
        y, sr = librosa.load(str(audio_path), sr=16000)
        
        # MFCC (26 features)
        mfcc = librosa.feature.mfcc(
            y=y, sr=sr, n_mfcc=13,
            n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )
        mfcc_mean = np.mean(mfcc, axis=1)
        mfcc_std = np.std(mfcc, axis=1)
        
        # Delta MFCC (26 features)
        delta_mfcc = librosa.feature.delta(mfcc)
        delta_mfcc_mean = np.mean(delta_mfcc, axis=1)
        delta_mfcc_std = np.std(delta_mfcc, axis=1)
        
        # Delta2 MFCC (26 features)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        delta2_mfcc_mean = np.mean(delta2_mfcc, axis=1)
        delta2_mfcc_std = np.std(delta2_mfcc, axis=1)
        
        # Mel spectrogram for energy features
        mel_spec = librosa.feature.melspectrogram(
            y=y, sr=sr, n_mels=24,
            n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )
        
        # Bark Energy (2 features)
        # NOTE: axis=0 means sum across mel bands (24 bins) → total energy per time frame
        # This represents the total spectral energy at each time point
        # Alternative: axis=1 would give energy per mel band (summed across time)
        bark_energy = np.sum(mel_spec, axis=0)  # Shape: (num_frames,)
        bark_energy_mean = np.mean(bark_energy)
        bark_energy_std = np.std(bark_energy)
        
        # Delta Energy (2 features)
        delta_energy = librosa.feature.delta(mel_spec)
        delta_energy_mean = np.mean(delta_energy)
        delta_energy_std = np.std(delta_energy)
        
        # Delta2 Energy (2 features)
        delta2_energy = librosa.feature.delta(mel_spec, order=2)
        delta2_energy_mean = np.mean(delta2_energy)
        delta2_energy_std = np.std(delta2_energy)
        
        # Spectral features (6 features)
        spectral_centroid = librosa.feature.spectral_centroid(
            y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(
            y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )[0]
        spectral_rolloff = librosa.feature.spectral_rolloff(
            y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )[0]
        
        # ZCR (2 features)
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        
        # Chroma (2 features)
        chroma = librosa.feature.chroma_stft(
            y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, win_length=win_length,
            window=window, center=center
        )
        
        # Concatenate all features
        features = np.concatenate([
            mfcc_mean, mfcc_std,                    # 26
            delta_mfcc_mean, delta_mfcc_std,        # 26
            delta2_mfcc_mean, delta2_mfcc_std,      # 26
            [bark_energy_mean, bark_energy_std],    # 2
            [delta_energy_mean, delta_energy_std],  # 2
            [delta2_energy_mean, delta2_energy_std],# 2
            [np.mean(spectral_centroid), np.std(spectral_centroid)],  # 2
            [np.mean(spectral_bandwidth), np.std(spectral_bandwidth)],# 2
            [np.mean(spectral_rolloff), np.std(spectral_rolloff)],    # 2
            [np.mean(zcr), np.std(zcr)],            # 2
            [np.mean(np.mean(chroma, axis=1)), np.std(np.mean(chroma, axis=1))]  # 2
        ])
        
        # Verify feature count
        assert len(features) == 94, f"Expected 94 features, got {len(features)}"
        
        logger.log(f"✓ Audio features extracted: {len(features)} features")
        return features
        
    except Exception as e:
        logger.log_error(f"Failed to extract audio features: {str(e)}")
        import traceback
        logger.log_error(traceback.format_exc())
        return None


def create_audio_feature_names() -> List[str]:
    """Create column names for 94 audio features"""
    feature_names = []
    
    # MFCC (26)
    for i in range(13):
        feature_names.append(f'mfcc{i+1}_mean')
        feature_names.append(f'mfcc{i+1}_std')
    
    # Delta MFCC (26)
    for i in range(13):
        feature_names.append(f'delta_mfcc{i+1}_mean')
        feature_names.append(f'delta_mfcc{i+1}_std')
    
    # Delta2 MFCC (26)
    for i in range(13):
        feature_names.append(f'delta2_mfcc{i+1}_mean')
        feature_names.append(f'delta2_mfcc{i+1}_std')
    
    # Energy features (6)
    feature_names.extend([
        'bark_energy_mean', 'bark_energy_std',
        'delta_energy_mean', 'delta_energy_std',
        'delta2_energy_mean', 'delta2_energy_std'
    ])
    
    # Spectral features (6)
    feature_names.extend([
        'spectral_centroid_mean', 'spectral_centroid_std',
        'spectral_bandwidth_mean', 'spectral_bandwidth_std',
        'spectral_rolloff_mean', 'spectral_rolloff_std'
    ])
    
    # ZCR (2)
    feature_names.extend(['zcr_mean', 'zcr_std'])
    
    # Chroma (2)
    feature_names.extend(['chroma_mean', 'chroma_std'])
    
    # Verify count
    assert len(feature_names) == 94, f"Expected 94 feature names, got {len(feature_names)}"
    
    return feature_names


# ==================== AUDIO QUALITY ====================
def check_audio_quality(audio_path: Path, config: I3DConfig, logger: I3DLogger) -> Dict:
    """Check audio quality with explicit STFT parameters"""
    try:
        # Get librosa parameters
        librosa_params = config.get('audio.librosa_params', {})
        n_fft = librosa_params.get('n_fft', 2048)
        hop_length = librosa_params.get('hop_length', 512)
        
        y, sr = librosa.load(str(audio_path), sr=16000)
        
        rms = librosa.feature.rms(y=y)[0]
        rms_mean = np.mean(rms)
        rms_std = np.std(rms)
        
        zcr = librosa.feature.zero_crossing_rate(y)[0]
        zcr_mean = np.mean(zcr)
        
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
        sc_mean = np.mean(spectral_centroid)
        
        # STFT with explicit parameters
        S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))
        noise_floor = np.percentile(S, 10)
        signal_power = np.mean(S ** 2)
        noise_power = noise_floor ** 2
        
        if noise_power > 0:
            snr = 10 * np.log10(signal_power / noise_power)
        else:
            snr = 60
        
        duration = len(y) / sr
        dynamic_range = np.max(np.abs(y)) - np.min(np.abs(y))
        
        rms_score = min(rms_mean / 0.1, 1.0)
        snr_score = min(max(snr - 10, 0) / 40, 1.0)
        zcr_score = 1 - min(zcr_mean / 0.2, 1.0)
        sc_score = min(sc_mean / 2000, 1.0)
        duration_score = min(duration / 5.0, 1.0)
        dr_score = min(dynamic_range / 0.5, 1.0)
        
        quality_score = (
            0.25 * rms_score + 0.25 * snr_score + 0.15 * zcr_score +
            0.15 * sc_score + 0.10 * duration_score + 0.10 * dr_score
        )
        
        with wave.open(str(audio_path), 'rb') as wf:
            n_channels = wf.getnchannels()
            sampwidth = wf.getsampwidth()
            framerate = wf.getframerate()
        
        return {
            'channels': n_channels,
            'sample_width': sampwidth,
            'frame_rate': framerate,
            'duration': duration,
            'rms': float(rms_mean),
            'rms_std': float(rms_std),
            'max_amplitude': float(np.max(np.abs(y))),
            'snr': float(snr),
            'snr_computation': f'STFT(n_fft={n_fft}, hop_length={hop_length})',
            'zcr': float(zcr_mean),
            'spectral_centroid': float(sc_mean),
            'dynamic_range': float(dynamic_range),
            'quality_score': float(quality_score)
        }
        
    except Exception as e:
        logger.log_error(f"Failed to check audio quality: {str(e)}")
        return {
            'channels': 0, 'sample_width': 0, 'frame_rate': 0, 'duration': 0,
            'rms': 0, 'rms_std': 0, 'max_amplitude': 0, 'snr': 0,
            'snr_computation': 'failed',
            'zcr': 0, 'spectral_centroid': 0, 'dynamic_range': 0, 'quality_score': 0
        }


# ==================== TEXT ANALYSIS ====================
def analyze_text_content(text: str, language: str = 'id') -> Dict:
    """
    Analyze text for linguistic features
    
    NOTE: Sentiment analysis for Indonesian uses TextBlob (English-based)
    and should be considered as proxy only. See config note.
    """
    if not text or len(text.strip()) == 0:
        return {
            'sentiment': 0, 'subjectivity': 0, 'complexity': 0, 'word_count': 0,
            'char_count': 0, 'avg_word_length': 0, 'unique_words': 0, 'lexical_diversity': 0
        }
    
    words = text.lower().split()
    unique_words = set(words)
    word_count = len(words)
    char_count = len(text)
    
    lexical_diversity = len(unique_words) / word_count if word_count > 0 else 0
    avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0
    
    # Sentiment analysis (proxy for Indonesian)
    sentiment = 0
    subjectivity = 0
    complexity = 0
    
    if language == 'en':  # Only reliable for English
        try:
            from textblob import TextBlob
            blob = TextBlob(text)
            sentiment = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            
            sentences = blob.sentences
            avg_sentence_length = sum(len(sentence.words) for sentence in sentences) / len(sentences) if sentences else 0
            complexity = (avg_word_length * 0.5 + avg_sentence_length * 0.5) / 10
        except Exception as e:
            logging.warning(f"TextBlob analysis failed: {str(e)}")
    
    return {
        'sentiment': sentiment,
        'subjectivity': subjectivity,
        'complexity': complexity,
        'word_count': word_count,
        'char_count': char_count,
        'avg_word_length': avg_word_length,
        'unique_words': len(unique_words),
        'lexical_diversity': lexical_diversity
    }


# ==================== TRANSLATION ====================
def translate_text_with_cache(text: str, translation_cache: TranslationCacheManager,
                              decision_log: DecisionLogManager = None,
                              filename: str = None,
                              max_retries: int = 3, delay: int = 1, 
                              logger: Optional[I3DLogger] = None) -> Optional[str]:
    """Translate Indonesian to English with caching"""
    if not text or len(text.strip()) == 0:
        return ""
    
    # Check cache first
    cached_translation = translation_cache.get_translation(text)
    if cached_translation:
        if logger:
            logger.log(f"✓ Using CACHED TRANSLATION")
        
        if decision_log and filename:
            decision_log.log_translation(filename, from_cache=True, text_length=len(text))
        
        return cached_translation
    
    # Translate
    translator = GoogleTranslator(source='id', target='en')
    
    for attempt in range(max_retries):
        try:
            if len(text) > 4500:
                chunks = [text[i:i+4500] for i in range(0, len(text), 4500)]
                translated_chunks = []
                
                for chunk in chunks:
                    result = translator.translate(chunk)
                    translated_chunks.append(result)
                    time.sleep(0.5)
                
                translation = ' '.join(translated_chunks)
            else:
                translation = translator.translate(text)
            
            # Save to cache
            translation_cache.save_translation(text, translation)
            
            if decision_log and filename:
                decision_log.log_translation(filename, from_cache=False, text_length=len(text))
            
            return translation
                
        except Exception as e:
            if attempt < max_retries - 1:
                if logger:
                    logger.log_warning(f"Translation attempt {attempt + 1} failed: {str(e)}. Retrying...")
                time.sleep(delay * (attempt + 1))
            else:
                if logger:
                    logger.log_error(f"Translation failed after {max_retries} attempts: {str(e)}")
                return None
    
    return None


# ==================== UTILITY FUNCTIONS ====================
def get_valid_video_files(directory: Path) -> List[str]:
    """Get all valid video files"""
    if not directory.exists():
        return []
    
    valid_extensions = ('.mp4', '.mov', '.avi', '.mkv', '.flv', '.wmv', '.m4v')
    
    files = []
    for file in directory.iterdir():
        if file.is_file():
            if file.name.startswith('.') or file.name.startswith('._'):
                continue
            if file.name in ['.DS_Store', 'Thumbs.db', 'desktop.ini']:
                continue
            if file.suffix.lower() in valid_extensions:
                files.append(file.name)
    
    return sorted(files)


def convert_numpy_types(obj):
    """Convert NumPy types to Python native types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    else:
        return obj


# ==================== DATA DICTIONARY GENERATOR ====================
def generate_data_dictionary(paths: I3DPathManager) -> Path:
    """Generate comprehensive data dictionary for all output CSVs"""
    
    dictionary = []
    
    # Text features (Indonesian)
    dictionary.extend([
        {'column': 'filename', 'type': 'string', 'description': 'Video filename (contains label prefix LIE_/TRUTH_)', 'unit': '-', 'range': '-', 'missing_value_policy': 'N/A'},
        {'column': 'text_indonesian_original', 'type': 'string', 'description': 'Original transcript in Indonesian', 'unit': '-', 'range': '-', 'missing_value_policy': 'empty string if failed'},
        {'column': 'text_indonesian_normalized', 'type': 'string', 'description': 'Transcript with numbers converted to words', 'unit': '-', 'range': '-', 'missing_value_policy': 'empty string if failed'},
        {'column': 'label', 'type': 'integer', 'description': 'Ground truth label', 'unit': '-', 'range': '0=truth, 1=lie', 'missing_value_policy': 'N/A'},
        {'column': 'dataset', 'type': 'string', 'description': 'Dataset identifier', 'unit': '-', 'range': 'I3D', 'missing_value_policy': 'N/A'},
        {'column': 'transcript_source', 'type': 'string', 'description': 'Source of transcript', 'unit': '-', 'range': 'manual/stt/stt_ultra/failed', 'missing_value_policy': 'failed'},
        {'column': 'transcript_from_cache', 'type': 'boolean', 'description': 'Whether transcript loaded from cache', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'transcript_attempt', 'type': 'integer', 'description': 'STT attempt number that succeeded', 'unit': '-', 'range': '0-7', 'missing_value_policy': '0'},
        {'column': 'transcript_mode', 'type': 'string', 'description': 'Extraction mode', 'unit': '-', 'range': 'normal/ultra/manual', 'missing_value_policy': 'normal'},
        {'column': 'stt_energy_threshold', 'type': 'float', 'description': 'Energy threshold used in successful STT attempt', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'char_count_id', 'type': 'integer', 'description': 'Character count (Indonesian)', 'unit': 'characters', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'word_count_id', 'type': 'integer', 'description': 'Word count (Indonesian)', 'unit': 'words', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'sentiment_id', 'type': 'float', 'description': 'Sentiment polarity (PROXY ONLY - TextBlob is English-based, not computed for Indonesian)', 'unit': '-', 'range': '-1.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'subjectivity_id', 'type': 'float', 'description': 'Subjectivity score (PROXY ONLY, not computed for Indonesian)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'complexity_id', 'type': 'float', 'description': 'Text complexity score', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'lexical_diversity_id', 'type': 'float', 'description': 'Unique words / total words', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
    ])
    
    # Text features (English)
    dictionary.extend([
        {'column': 'text_english', 'type': 'string', 'description': 'English translation via Google Translate', 'unit': '-', 'range': '-', 'missing_value_policy': 'empty string if failed'},
        {'column': 'char_count_en', 'type': 'integer', 'description': 'Character count (English)', 'unit': 'characters', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'word_count_en', 'type': 'integer', 'description': 'Word count (English)', 'unit': 'words', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'sentiment_en', 'type': 'float', 'description': 'Sentiment polarity (English - TextBlob)', 'unit': '-', 'range': '-1.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'subjectivity_en', 'type': 'float', 'description': 'Subjectivity score (English)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'complexity_en', 'type': 'float', 'description': 'Text complexity score (English)', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'lexical_diversity_en', 'type': 'float', 'description': 'Unique words / total words (English)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
    ])
    
    # Number features
    dictionary.extend([
        {'column': 'has_numbers', 'type': 'boolean', 'description': 'Contains numeric expressions', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'number_count', 'type': 'integer', 'description': 'Count of numeric expressions', 'unit': 'count', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'has_vague_quantifiers', 'type': 'boolean', 'description': 'Contains vague quantifiers (sekitar, kira-kira, etc)', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'has_exact_quantifiers', 'type': 'boolean', 'description': 'Contains exact quantifiers (tepat, persis, etc)', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'number_word_ratio', 'type': 'float', 'description': 'Ratio of number words to total words', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'has_large_numbers', 'type': 'boolean', 'description': 'Contains numbers > 1000', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'has_decimal_numbers', 'type': 'boolean', 'description': 'Contains decimal numbers', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
    ])
    
    # Audio features (94 total)
    for i in range(1, 14):
        dictionary.extend([
            {'column': f'mfcc{i}_mean', 'type': 'float', 'description': f'MFCC coefficient {i} mean', 'unit': '-', 'range': 'real', 'missing_value_policy': '0'},
            {'column': f'mfcc{i}_std', 'type': 'float', 'description': f'MFCC coefficient {i} std deviation', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        ])
    
    for i in range(1, 14):
        dictionary.extend([
            {'column': f'delta_mfcc{i}_mean', 'type': 'float', 'description': f'Delta MFCC coefficient {i} mean', 'unit': '-', 'range': 'real', 'missing_value_policy': '0'},
            {'column': f'delta_mfcc{i}_std', 'type': 'float', 'description': f'Delta MFCC coefficient {i} std deviation', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        ])
    
    for i in range(1, 14):
        dictionary.extend([
            {'column': f'delta2_mfcc{i}_mean', 'type': 'float', 'description': f'Delta-delta MFCC coefficient {i} mean', 'unit': '-', 'range': 'real', 'missing_value_policy': '0'},
            {'column': f'delta2_mfcc{i}_std', 'type': 'float', 'description': f'Delta-delta MFCC coefficient {i} std deviation', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        ])
    
    # ✅ PERBAIKAN: Bark Energy description lebih spesifik
    dictionary.extend([
        {
            'column': 'bark_energy_mean', 
            'type': 'float', 
            'description': (
                'Bark energy mean. Computed as sum of 24 mel-band energies per time frame '
                '(axis=0 summation across mel bins), then averaged across all time frames. '
                'Represents total spectral energy per time point. '
                'See Methods section for mel spectrogram parameters (n_mels=24, n_fft=2048, hop_length=512).'
            ), 
            'unit': '-', 
            'range': '>=0', 
            'missing_value_policy': '0'
        },
        {
            'column': 'bark_energy_std', 
            'type': 'float', 
            'description': 'Bark energy std deviation (temporal variation across time frames)', 
            'unit': '-', 
            'range': '>=0', 
            'missing_value_policy': '0'
        },
        {
            'column': 'delta_energy_mean', 
            'type': 'float', 
            'description': 'Delta energy mean (first-order temporal derivative of mel spectrogram energy)', 
            'unit': '-', 
            'range': 'real', 
            'missing_value_policy': '0'
        },
        {
            'column': 'delta_energy_std', 
            'type': 'float', 
            'description': 'Delta energy std deviation', 
            'unit': '-', 
            'range': '>=0', 
            'missing_value_policy': '0'
        },
        {
            'column': 'delta2_energy_mean', 
            'type': 'float', 
            'description': 'Delta-delta energy mean (second-order temporal derivative)', 
            'unit': '-', 
            'range': 'real', 
            'missing_value_policy': '0'
        },
        {
            'column': 'delta2_energy_std', 
            'type': 'float', 
            'description': 'Delta-delta energy std deviation', 
            'unit': '-', 
            'range': '>=0', 
            'missing_value_policy': '0'
        },
        {'column': 'spectral_centroid_mean', 'type': 'float', 'description': 'Spectral centroid mean', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'spectral_centroid_std', 'type': 'float', 'description': 'Spectral centroid std deviation', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'spectral_bandwidth_mean', 'type': 'float', 'description': 'Spectral bandwidth mean', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'spectral_bandwidth_std', 'type': 'float', 'description': 'Spectral bandwidth std deviation', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'spectral_rolloff_mean', 'type': 'float', 'description': 'Spectral rolloff mean', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'spectral_rolloff_std', 'type': 'float', 'description': 'Spectral rolloff std deviation', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'zcr_mean', 'type': 'float', 'description': 'Zero-crossing rate mean', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'zcr_std', 'type': 'float', 'description': 'Zero-crossing rate std deviation', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'chroma_mean', 'type': 'float', 'description': 'Chroma feature mean', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'chroma_std', 'type': 'float', 'description': 'Chroma feature std deviation', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
    ])
    
    # Audio quality
    dictionary.extend([
        {'column': 'audio_quality_score', 'type': 'float', 'description': 'Overall audio quality score', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'audio_duration', 'type': 'float', 'description': 'Audio duration', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'audio_snr', 'type': 'float', 'description': 'Signal-to-noise ratio (STFT-based, see Methods)', 'unit': 'dB', 'range': 'real', 'missing_value_policy': '0'},
        {'column': 'audio_snr_computation', 'type': 'string', 'description': 'SNR computation method with parameters', 'unit': '-', 'range': '-', 'missing_value_policy': 'failed'},
        {'column': 'audio_rms', 'type': 'float', 'description': 'Root mean square amplitude', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'audio_spectral_centroid', 'type': 'float', 'description': 'Mean spectral centroid', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'audio_dynamic_range', 'type': 'float', 'description': 'Dynamic range', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
    ])
    
    # Pause features (17)
    dictionary.extend([
        {'column': 'pause_num_pauses', 'type': 'integer', 'description': 'Number of pauses detected', 'unit': 'count', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_total_pause_duration', 'type': 'float', 'description': 'Total pause duration', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_avg_pause_duration', 'type': 'float', 'description': 'Average pause duration', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_longest_pause_duration', 'type': 'float', 'description': 'Longest pause duration', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_pause_frequency', 'type': 'float', 'description': 'Pauses per second', 'unit': 'Hz', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_speech_duration', 'type': 'float', 'description': 'Speech duration (excluding pauses)', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_speech_rate', 'type': 'float', 'description': 'Speech rate (speech / total duration)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'pause_pause_ratio', 'type': 'float', 'description': 'Pause ratio (pause / total duration)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'pause_early_pauses', 'type': 'integer', 'description': 'Pauses in first third', 'unit': 'count', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_middle_pauses', 'type': 'integer', 'description': 'Pauses in middle third', 'unit': 'count', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_late_pauses', 'type': 'integer', 'description': 'Pauses in last third', 'unit': 'count', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_pause_std', 'type': 'float', 'description': 'Pause duration std deviation', 'unit': 'seconds', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_pause_variability', 'type': 'float', 'description': 'Pause variability (std / mean)', 'unit': '-', 'range': '>=0', 'missing_value_policy': '0'},
        {'column': 'pause_hesitation_score', 'type': 'float', 'description': 'Composite hesitation score', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0'},
        {'column': 'pause_has_long_pauses', 'type': 'boolean', 'description': 'Has pauses > 1.5s', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'pause_has_frequent_pauses', 'type': 'boolean', 'description': 'Pause frequency > 1.0 Hz', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'pause_has_high_pause_ratio', 'type': 'boolean', 'description': 'Pause ratio > 0.3', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
    ])
    
    # Landmark features
    dictionary.extend([
        {'column': 'Video_Name', 'type': 'string', 'description': 'Video identifier (without extension)', 'unit': '-', 'range': '-', 'missing_value_policy': 'N/A'},
        {'column': 'Frame', 'type': 'integer', 'description': 'Frame number', 'unit': 'frame', 'range': '>=0', 'missing_value_policy': 'N/A'},
    ])
    
    # Face/Iris landmarks (478 = 468 face + 10 iris)
    for i in range(478):
        dictionary.extend([
            {'column': f'Landmark_{i}_X', 'type': 'float', 'description': f'Face/Iris landmark {i} X coordinate (normalized 0-1)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0.0 if not detected'},
            {'column': f'Landmark_{i}_Y', 'type': 'float', 'description': f'Face/Iris landmark {i} Y coordinate (normalized 0-1)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0.0 if not detected'},
            {'column': f'Landmark_{i}_Z', 'type': 'float', 'description': f'Face/Iris landmark {i} Z coordinate (depth, relative scale)', 'unit': '-', 'range': 'real', 'missing_value_policy': '0.0 if not detected'},
        ])
    
    # Pose landmarks (33)
    for i in range(33):
        dictionary.extend([
            {'column': f'Pose_{i}_X', 'type': 'float', 'description': f'Pose landmark {i} X coordinate (normalized 0-1)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0.0 if not detected'},
            {'column': f'Pose_{i}_Y', 'type': 'float', 'description': f'Pose landmark {i} Y coordinate (normalized 0-1)', 'unit': '-', 'range': '0.0 to 1.0', 'missing_value_policy': '0.0 if not detected'},
            {'column': f'Pose_{i}_Z', 'type': 'float', 'description': f'Pose landmark {i} Z coordinate (depth, relative scale)', 'unit': '-', 'range': 'real', 'missing_value_policy': '0.0 if not detected'},
        ])
    
    dictionary.append({'column': 'Class', 'type': 'integer', 'description': 'Ground truth label', 'unit': '-', 'range': '0=truth, 1=lie', 'missing_value_policy': 'N/A'})
    
    # Re-extraction metadata
    dictionary.extend([
        {'column': 'reextraction_triggered', 'type': 'boolean', 'description': 'Whether re-extraction was triggered for this file', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'reextraction_success', 'type': 'boolean', 'description': 'Whether re-extraction succeeded', 'unit': '-', 'range': 'true/false', 'missing_value_policy': 'false'},
        {'column': 'reextraction_params', 'type': 'string', 'description': 'JSON string of re-extraction parameters', 'unit': '-', 'range': 'JSON', 'missing_value_policy': 'empty string'},
    ])
    
    # Save to CSV
    df_dict = pd.DataFrame(dictionary)
    dict_path = paths.paths['metadata'] / 'data_dictionary.csv'
    df_dict.to_csv(dict_path, index=False, encoding='utf-8')
    
    # ✅ NEW: Schema validation check
    _validate_data_dictionary_schema(df_dict, paths)
    
    return dict_path


def _validate_data_dictionary_schema(df_dict: pd.DataFrame, paths: I3DPathManager):
    """
    Validate that data dictionary matches actual dataset columns.
    Saves validation report to validation folder.
    """
    validation_results = {
        'timestamp': datetime.now().isoformat(),
        'dictionary_columns': df_dict['column'].tolist(),
        'datasets_checked': {},
        'issues': []
    }
    
    # Check against actual datasets (if they exist)
    datasets_to_check = {
        'TextDataset_Indonesian': paths.paths['text'] / 'TextDataset_Indonesian.csv',
        'TextDataset_English': paths.paths['text'] / 'TextDataset_English.csv',
        'AudioDataset_Features': paths.paths['audio'] / 'AudioDataset_Features.csv',
        'PauseFeatures': paths.paths['audio'] / 'PauseFeatures.csv',
        'LandmarkDataset': paths.paths['visual'] / 'LandmarkDataset.csv',
        'MultimodalDataset_Full': paths.paths['multimodal'] / 'MultimodalDataset_Full.csv',
    }
    
    dict_columns = set(df_dict['column'].tolist())
    
    for dataset_name, dataset_path in datasets_to_check.items():
        if not dataset_path.exists():
            continue
        
        try:
            # Read only header
            df_dataset = pd.read_csv(dataset_path, nrows=0)
            dataset_columns = set(df_dataset.columns)
            
            # Check for missing columns in dictionary
            missing_in_dict = dataset_columns - dict_columns
            # Check for extra columns in dictionary (not in dataset)
            extra_in_dict = dict_columns - dataset_columns
            
            validation_results['datasets_checked'][dataset_name] = {
                'path': str(dataset_path),
                'column_count': len(dataset_columns),
                'missing_in_dictionary': list(missing_in_dict),
                'extra_in_dictionary': list(extra_in_dict),
                'status': 'OK' if not missing_in_dict and not extra_in_dict else 'MISMATCH'
            }
            
            if missing_in_dict:
                validation_results['issues'].append({
                    'dataset': dataset_name,
                    'issue': 'missing_in_dictionary',
                    'columns': list(missing_in_dict)
                })
            
            if extra_in_dict:
                validation_results['issues'].append({
                    'dataset': dataset_name,
                    'issue': 'extra_in_dictionary',
                    'columns': list(extra_in_dict)
                })
        
        except Exception as e:
            validation_results['datasets_checked'][dataset_name] = {
                'path': str(dataset_path),
                'error': str(e),
                'status': 'ERROR'
            }
    
    # Save validation report
    validation_path = paths.paths['validation'] / 'schema_check.json'
    with open(validation_path, 'w', encoding='utf-8') as f:
        json.dump(validation_results, f, indent=2, ensure_ascii=False)
    
    # Log summary
    if validation_results['issues']:
        logging.warning(f"⚠️ Data dictionary schema issues found: {len(validation_results['issues'])}")
        logging.warning(f"   See: {validation_path}")
    else:
        logging.info(f"✓ Data dictionary schema validated: No issues")


# ==================== RUN MANIFEST GENERATOR ====================
def generate_run_manifest(env_info: Dict, config: I3DConfig, paths: I3DPathManager, 
                         stats: Dict, video_files: List[str]) -> Path:
    """Generate comprehensive run manifest for reproducibility"""
    
    deterministic_mode = config.get('speech_recognition.deterministic_mode', False)
    
    manifest = {
        'run_info': {
            'timestamp': datetime.now().isoformat(),
            'pipeline_version': VERSION,
            'pipeline_name': PIPELINE_NAME,
            'total_files_processed': stats.get('total_files', 0),
            'success_rate': f"{stats.get('successful_text_conversions', 0) / max(stats.get('total_files', 1), 1) * 100:.1f}%"
        },
        'environment': env_info,
        'configuration': config.config,
        'input_files': {
            'lie_videos': [f for f in video_files if f.startswith('LIE_')],
            'truth_videos': [f for f in video_files if f.startswith('TRUTH_')],
            'total_count': len(video_files)
        },
        'processing_statistics': convert_numpy_types(stats),
        'determinism_notes': {
            'transcript_caching': 'Enabled - transcripts frozen after first extraction',
            'translation_caching': 'Enabled - MD5-based cache for Google Translate',
            'audio_enhancement': 'Fixed parameters based on dBFS thresholds',
            'stt_deterministic_mode': deterministic_mode,
            'stt_dynamic_energy_threshold': 'Disabled' if deterministic_mode else 'Enabled (default)',
            'librosa_parameters': config.get('audio.librosa_params', {}),
            'mediapipe_landmarks': {
                'face_mesh_refine_landmarks': True,
                'total_face_iris_landmarks': 478,
                'breakdown': '468 face + 10 iris',
                'pose_landmarks': 33,
                'validation': 'Runtime validation checks landmark count per frame',
                'mediapipe_version': env_info['packages'].get('mediapipe', 'unknown')
            },
            'external_services': [
                'Google Web Speech API (for STT - non-deterministic on first run)',
                'Google Translate API (for translation - non-deterministic on first run)'
            ],
            'reproducibility_statement': (
                'Pipeline is deterministic ONLY for cached files. '
                'First run uses external APIs (Google STT + Translate) which are non-deterministic. '
                'Subsequent runs with same cache produce identical output for cached files. '
                'New files remain non-deterministic until cached. '
                'For full reproducibility: (1) Keep cache directories intact, (2) Set force_retranscribe=false, '
                '(3) Set deterministic_mode=true for STT (disables dynamic thresholds).'
            )
        },
        'data_quality_notes': {
            'manual_transcripts': f"{stats.get('manual_transcripts_used', 0)} files use manually verified transcripts",
            'sentiment_analysis_caveat': 'TextBlob sentiment for Indonesian is proxy only (English-based model). Use sentiment_en for reliable sentiment analysis.',
            'landmark_missing_values': 'Missing landmarks filled with (0.0, 0.0, 0.0). Do NOT interpret as actual position. Filter or impute before analysis.',
            'filename_contains_label': 'WARNING: Filenames contain label prefix (LIE_/TRUTH_) - MUST remove before ML training to avoid label leakage',
            'bark_energy_definition': 'Bark energy computed as sum across mel bands (axis=0) → total energy per time frame. See data dictionary for details.'
        },
        'ethics_and_privacy': {
            'data_type': 'Interrogation videos with audio, facial landmarks, and transcripts',
            'contains_personal_data': True,
            'data_components': [
                'Video frames (not included in derived features)',
                'Audio waveforms (not included in derived features)',
                'Facial landmarks (478 points including iris)',
                'Pose landmarks (33 body keypoints)',
                'Speech transcripts (Indonesian and English)',
                'Audio features (94 acoustic features)'
            ],
            'public_release_policy': 'Derived features (landmarks, audio features, transcripts) may be released. Raw video/audio require additional consent/anonymization.',
            'recommendation': 'Consult IRB/ethics board for your institution regarding public release of interrogation data'
        },
        'output_files': {
            'text_indonesian': str(paths.paths['text'] / 'TextDataset_Indonesian.csv'),
            'text_english': str(paths.paths['text'] / 'TextDataset_English.csv'),
            'number_features': str(paths.paths['text'] / 'NumberFeatures.csv'),
            'audio_features': str(paths.paths['audio'] / 'AudioDataset_Features.csv'),
            'pause_features': str(paths.paths['audio'] / 'PauseFeatures.csv'),
            'landmarks': str(paths.paths['visual'] / 'LandmarkDataset.csv'),
            'multimodal_full': str(paths.paths['multimodal'] / 'MultimodalDataset_Full.csv'),
            'publication_ready': str(paths.paths['multimodal'] / 'PublicationDataset.csv'),
            'data_dictionary': str(paths.paths['metadata'] / 'data_dictionary.csv'),
            'decision_log': str(paths.paths['decision_logs'] / 'decision_log.jsonl'),
            'transcript_cache': str(paths.paths['cache'] / 'transcripts'),
            'translation_cache': str(paths.paths['cache'] / 'translation_cache.json'),
            'schema_validation': str(paths.paths['validation'] / 'schema_check.json')
        },
        'citation': config.get('dataset.citation', 'Please cite: [Your Paper]'),
        'license': 'See LICENSE file or paper for data usage terms',
        'contact': 'See paper for contact information'
    }
    
    # Save manifest
    manifest_path = paths.paths['metadata'] / 'run_manifest.json'
    with open(manifest_path, 'w', encoding='utf-8') as f:
        json.dump(manifest, f, indent=2, ensure_ascii=False)
    
    return manifest_path


# ==================== PROCESSING SUMMARY GENERATOR ====================
def generate_processing_summary(text_audio_data: List[Dict], paths: I3DPathManager) -> Path:
    """Generate per-file processing summary for Technical Validation"""
    
    summary_data = []
    
    for entry in text_audio_data:
        summary = {
            'filename': entry.get('filename', ''),
            'label': entry.get('label', -1),
            'transcript_source': entry.get('transcript_source', 'unknown'),
            'transcript_from_cache': entry.get('transcript_from_cache', False),
            'transcript_attempt': entry.get('transcript_attempt', 0),
            'stt_energy_threshold': entry.get('stt_energy_threshold', 0.0),
            'text_length': len(entry.get('text_indonesian_normalized', '')),
            'audio_duration': entry.get('audio_duration', 0),
            'audio_quality_score': entry.get('audio_quality_score', 0),
            'audio_snr': entry.get('audio_snr', 0),
            'has_numbers': entry.get('has_numbers', False),
            'pause_count': entry.get('pause_num_pauses', 0),
            'hesitation_score': entry.get('pause_hesitation_score', 0),
            'reextraction_triggered': entry.get('reextraction_triggered', False),
            'reextraction_success': entry.get('reextraction_success', False),
            'processing_success': entry.get('transcript_source', 'failed') != 'failed'
        }
        summary_data.append(summary)
    
    df_summary = pd.DataFrame(summary_data)
    summary_path = paths.paths['quality_reports'] / 'processing_summary.csv'
    df_summary.to_csv(summary_path, index=False, encoding='utf-8')
    
    return summary_path


# ==================== MARKDOWN REPORT (UPDATED) ====================
def generate_markdown_report(stats: Dict, output_paths: Dict, paths: I3DPathManager, 
                            start_time: datetime, end_time: datetime, 
                            env_info: Dict, config: I3DConfig) -> Path:
    """Generate comprehensive markdown report for Scientific Data"""
    duration = end_time - start_time
    hours, remainder = divmod(duration.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    
    manual_count = stats.get('manual_transcripts_used', 0)
    cached_transcripts = stats.get('cached_transcripts_used', 0)
    cached_translations = stats.get('cached_translations_used', 0)
    deterministic_mode = config.get('speech_recognition.deterministic_mode', False)
    
    markdown_content = f"""# 📊 I3D Dataset Extraction Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  
**Version:** {VERSION} - {PIPELINE_NAME}  
**Language:** Indonesian  
**Status:** ✅ Production-Ready for Scientific Data Submission

---

## ⚠️ CRITICAL NOTES FOR SCIENTIFIC DATA REVIEWERS

### 1. Determinism Statement
- **First Run**: Uses external APIs (Google STT + Translate) → **non-deterministic**
- **Subsequent Runs (with cache)**: All cached outputs → **deterministic for cached files**
- **Reproducibility**: Same input + same cache → identical output for cached files
- **New Files**: Will use external APIs → non-deterministic until cached

### 2. External Services Used
- **Google Web Speech API**: Speech-to-text (non-deterministic, results cached per file)
- **Google Translate API**: Translation (non-deterministic, MD5-cached per text)
- **Mitigation**: All outputs frozen in cache and provided in dataset repository

### 3. Known Limitations
- **Sentiment Analysis**: TextBlob for Indonesian is **proxy only** (English-based model). Use `sentiment_en` for reliable analysis.
- **Filename Leakage**: Filenames contain label prefix (`LIE_`/`TRUTH_`) - **MUST remove before ML training**
- **Missing Landmarks**: Filled with `(0.0, 0.0, 0.0)` - **NOT actual position**. Filter or impute before analysis.
- **Manual Transcripts**: {manual_count} files use manually verified transcripts (documented in cache)

### 4. Audio Features Verification
- **Claimed**: 94 features ✅
- **Actual**: 94 features ✅ (verified with assert)
- **Breakdown**: MFCC(26) + ΔMFCC(26) + Δ²MFCC(26) + Energy(6) + Spectral(6) + ZCR(2) + Chroma(2) = 94

### 5. MediaPipe Landmarks Verification
- **Face Mesh**: 478 landmarks ✅ (468 face + 10 iris with `refine_landmarks=True`)
- **Pose**: 33 landmarks ✅
- **Total per frame**: 478×3 + 33×3 = 1533 coordinate values + metadata

### 6. Ethics & Privacy
- **Data Type**: Interrogation videos with identifiable faces and voices
- **Public Release**: Derived features (landmarks, audio features, transcripts) may be released
- **Raw Data**: Video/audio require additional consent or anonymization
- **Recommendation**: Consult IRB/ethics board before public release

---

## ✅ Extraction Summary

| Metric | Value |
|--------|-------|
| **Total Files** | {stats['total_files']} |
| **Text Extraction** | {stats['successful_text_conversions']}/{stats['total_files']} ({stats['successful_text_conversions']/max(stats['total_files'], 1)*100:.1f}% success) |
| **Manual Transcripts Used** | {manual_count} |
| **Cached Transcripts Used** | {cached_transcripts} |
| **Audio Extraction** | {stats['successful_audio_extractions']}/{stats['total_files']} ({stats['successful_audio_extractions']/max(stats['total_files'], 1)*100:.1f}% success) |
| **Translation** | {stats['successful_translations']}/{stats['successful_text_conversions']} ({stats['successful_translations']/max(stats['successful_text_conversions'], 1)*100:.1f}% success) |
| **Cached Translations Used** | {cached_translations} |
| **Landmark Extraction** | {stats['successful_landmark_extractions']}/{stats['total_files']} ({stats['successful_landmark_extractions']/max(stats['total_files'], 1)*100:.1f}% success) |
| **Re-extractions** | {stats['reextraction_triggered']} triggered, {stats['reextraction_successful']} successful |

---

## ⏱️ Processing Time

| Metric | Value |
|--------|-------|
| **Start Time** | {start_time.strftime('%Y-%m-%d %H:%M:%S')} |
| **End Time** | {end_time.strftime('%Y-%m-%d %H:%M:%S')} |
| **Total Duration** | {int(hours)}h {int(minutes)}m {int(seconds)}s |

---

## 🖥️ Environment (Reproducibility)

| Component | Version |
|-----------|---------|
| **Python** | {env_info['python_version'].split()[0]} |
| **Platform** | {env_info['platform']['system']} {env_info['platform']['release']} |
| **NumPy** | {env_info['packages'].get('numpy', 'N/A')} |
| **Librosa** | {env_info['packages'].get('librosa', 'N/A')} |
| **MediaPipe** | {env_info['packages'].get('mediapipe', 'N/A')} |
| **OpenCV** | {env_info['packages'].get('opencv-python', 'N/A')} |
| **FFmpeg** | {env_info.get('ffmpeg_version', 'N/A')[:50]}... |

**Full environment**: See `run_manifest.json`

---

## 📊 Detection Statistics

| Metric | Value |
|--------|-------|
| **Total Frames Processed** | {stats['total_frames_processed']:,} |
| **Face Detection** | {stats['face_detected']:,} frames ({stats['face_detected']/max(stats['total_frames_processed'],1)*100:.1f}%) |
| **Iris Detection** | {stats['iris_detected']:,} frames ({stats['iris_detected']/max(stats['total_frames_processed'],1)*100:.1f}%) |
| **Pose Detection** | {stats['pose_detected']:,} frames ({stats['pose_detected']/max(stats['total_frames_processed'],1)*100:.1f}%) |

---

## 🔢 Number Processing

| Metric | Value |
|--------|-------|
| **Texts with Numbers** | {stats['texts_with_numbers']} |
| **Numbers Normalized** | {stats['numbers_normalized']} |
| **Normalization Range** | 0 to Trillion (Indonesian words) |

---

## 🔒 Deterministic Processing Details

### Transcript Freezing
✅ **{cached_transcripts} transcripts loaded from cache**  
- Cache location: `dataset/_cache/I3D/transcripts/`
- Format: JSON per file (text, source, attempt, mode, stt_energy_threshold, timestamp)
- Priority: Cache → Manual → STT
- Override: Set `force_retranscribe: true` in config

### Translation Freezing
✅ **{cached_translations} translations loaded from cache**  
- Cache file: `dataset/_cache/I3D/translation_cache.json`
- Key system: MD5 hash of source text
- Eliminates Google Translate API variability

### Fixed Audio Enhancement
✅ **Deterministic parameters**  
- Boost amounts: 40dB (ultra), 35dB (aggressive), 20dB (moderate)
- Target dBFS: -18 (fixed)
- Compression ratio: 6.0 (normal), 10.0 (ultra)
- All thresholds fixed (not adaptive)

### Librosa Parameters (Explicit)
```yaml
n_fft: 2048
hop_length: 512
win_length: 2048
window: hann
center: true
pad_mode: constant
STT Configuration
Deterministic Mode: {'Enabled' if deterministic_mode else 'Disabled (default)'}
Dynamic Energy Threshold: {'Disabled' if deterministic_mode else 'Enabled'}
Energy thresholds logged: Yes (per transcript in cache)
Note: Deterministic mode disables adjust_for_ambient_noise() and dynamic_energy_threshold
MediaPipe Configuration
face_mesh:
  refine_landmarks: true  # Enables 10 iris landmarks
  total_landmarks: 478    # 468 face + 10 iris
pose:
  model_complexity: 1
  total_landmarks: 33
Decision Logging
✅ All processing decisions logged

Log file: dataset/_logs/I3D/decisions/decision_log.jsonl
Includes: re-extraction triggers, transcript sources, translations, energy thresholds
Format: JSONL (one decision per line)
📝 Manual Transcripts
✅ {manual_count} files use manually verified transcripts

Automatically applied for known problematic files
Ensures 100% accuracy for difficult audio
Source documented in transcript metadata
List: See MANUAL_TRANSCRIPTS dictionary in code
🔄 Resume Capability
✅ Checkpoint system enabled

Auto-save every 5 files
Resume from last checkpoint if interrupted
Preserves all cache states
Checkpoint location: dataset/_checkpoints/I3D/
📦 Output Files
Text Datasets
Indonesian Text: {output_paths.get('text_indonesian', 'N/A')}
English Text: {output_paths.get('text_english', 'N/A')}
Number Features: {output_paths.get('number_features', 'N/A')}
Audio Datasets
Audio Features (94): {output_paths.get('audio_features', 'N/A')}
Pause Features (17): {output_paths.get('pause_features', 'N/A')}
Visual Datasets
Landmarks (478 face + 33 pose): {output_paths.get('landmark', 'N/A')}
Multimodal Datasets
Full Dataset: {output_paths.get('multimodal_full', 'N/A')}
Publication Ready: {output_paths.get('publication', 'N/A')}
Metadata & Validation
Data Dictionary: dataset/metadata/I3D/data_dictionary.csv
Run Manifest: dataset/metadata/I3D/run_manifest.json
Processing Summary: dataset/validation/I3D/quality_reports/processing_summary.csv
Decision Log: dataset/_logs/I3D/decisions/decision_log.jsonl
Transcript Cache: dataset/_cache/I3D/transcripts/
Translation Cache: dataset/_cache/I3D/translation_cache.json
🎯 Features Extracted
Text Features (per language)
Character count
Word count
Sentiment analysis (⚠️ Indonesian: proxy only, use English version)
Subjectivity score
Complexity score
Lexical diversity
Number Features (7)
Has numbers (boolean)
Number count
Has vague quantifiers
Has exact quantifiers
Number-to-word ratio
Has large numbers (>1000)
Has decimal numbers
Audio Features (94) ✅ VERIFIED
Breakdown:

MFCC: 13 coefficients × 2 stats (mean, std) = 26
Delta MFCC: 13 × 2 = 26
Delta-Delta MFCC: 13 × 2 = 26
Bark Energy: 2 (mean, std)
Delta Energy: 2
Delta-Delta Energy: 2
Spectral Centroid: 2 (mean, std)
Spectral Bandwidth: 2
Spectral Rolloff: 2
Zero-Crossing Rate: 2
Chroma: 2
Total: 26 + 26 + 26 + 2 + 2 + 2 + 2 + 2 + 2 + 2 + 2 = 94 features ✅
Pause Features (17)
Number of pauses
Total pause duration
Average pause duration
Longest pause duration
Pause frequency
Speech duration
Speech rate
Pause ratio
Early/middle/late pauses (3)
Pause standard deviation
Pause variability
Hesitation score
Boolean flags (3): long pauses, frequent pauses, high pause ratio
Visual Features ✅ VERIFIED
Face Mesh: 468 facial landmarks
Iris: 10 iris landmarks (indices 468-477)
Total Face/Iris: 478 landmarks
Pose: 33 body keypoints
Coordinates: 3D (x, y, z) for each landmark
Missing values: Filled with (0.0, 0.0, 0.0) - NOT actual position
⚙️ Processing Configuration
Audio Enhancement (Deterministic)
Sampling rate: 16000 Hz
Fixed boost thresholds: -55, -45, -35 dBFS
Fixed boost amounts: 40, 35, 20 dB
Target dBFS: -18 (fixed)
Silence threshold: -50 dBFS (fixed)
Compression ratio: 6.0 (normal), 10.0 (ultra)
Filters: Highpass 80Hz, Lowpass 3000Hz
Video Preprocessing
Multi-level zoom: 1.5x to 10x
CLAHE enhancement: clip_limit=4.0, tile_size=(4,4)
Gamma correction: 1.4
Unsharp masking: strength=1.8
Eye region enhancement: For iris detection
Speech Recognition
Language: Indonesian (id-ID)
Max attempts: 7 (ultra mode), 3 (normal mode)
Progressive energy threshold: 300 → 10
Dynamic pause threshold: 0.8 → 0.1
Transcript caching: ✅ Enabled
Manual transcript fallback: ✅ Enabled
Energy threshold logging: ✅ Per transcript
Librosa (Explicit Parameters)
n_fft = 2048
hop_length = 512
win_length = 2048
window = 'hann'
center = True
pad_mode = 'constant'
MediaPipe (Explicit Parameters)
face_mesh = FaceMesh(
    static_image_mode=False,
    max_num_faces=1,
    refine_landmarks=True,  # Enables iris (10 landmarks)
    min_detection_confidence=0.2,
    min_tracking_confidence=0.2
)

pose = Pose(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.3,
    min_tracking_confidence=0.3
)
📈 Quality Metrics
Audio Quality Checks
RMS (Root Mean Square)
SNR (Signal-to-Noise Ratio) - STFT-based with explicit n_fft=2048, hop_length=512
Zero-crossing rate
Spectral centroid
Dynamic range
Overall quality score (0.0 to 1.0)
Landmark Quality (per video)
Face detection rate
Iris detection rate
Pose detection rate
Strategy success distribution
⚠️ Failed Samples
Total Failed: {len(stats.get('errors', []))}

Failed samples saved in: dataset/processed/I3D/reextraction/failed_samples.csv

💡 Usage Notes for Researchers
1. Filename Label Leakage
⚠️ WARNING: Filenames contain label prefix (LIE_ / TRUTH_)

Before ML training:
# Option 1: Remove filename
df = df.drop(columns=['filename'])

# Option 2: Encode filename
import hashlib
df['filename_encoded'] = df['filename'].apply(
    lambda x: hashlib.md5(x.encode()).hexdigest()
)
2. Sentiment Analysis Caveat
⚠️ Indonesian sentiment scores are PROXY ONLY

TextBlob uses English-based model
For Indonesian, use only English translation sentiment (sentiment_en)
Or use Indonesian-specific sentiment model
3. Missing Landmark Values
⚠️ Missing landmarks filled with (0.0, 0.0, 0.0)

Do NOT interpret (0.0, 0.0, 0.0) as actual landmark position
Filter out or impute before analysis:
# Filter out missing landmarks
def is_valid_landmark(x, y, z):
    return not (x == 0.0 and y == 0.0 and z == 0.0)

# Or check if all landmarks in frame are missing
# Filter out missing landmarks
def is_valid_landmark(x, y, z):
    return not (x == 0.0 and y == 0.0 and z == 0.0)

# Or check if all landmarks in frame are missing
4. Re-running Pipeline
For deterministic results:

Keep cache directories intact
Set force_retranscribe: false in config
Same input + cache → identical output
For fresh extraction:

Delete cache directories
Or set force_retranscribe: true
Results will differ due to external APIs
📚 Documentation Files
For Scientific Data Submission
Data Dictionary: data_dictionary.csv - Complete column descriptions
Run Manifest: run_manifest.json - Full reproducibility info
Processing Summary: processing_summary.csv - Per-file quality metrics
Decision Log: decision_log.jsonl - All processing decisions
This Report: I3D_EXTRACTION_REPORT.md - Overview
For Methods Section
Librosa parameters: See run_manifest.json
STT configuration: See run_manifest.json
Enhancement parameters: See run_manifest.json
MediaPipe configuration: See run_manifest.json
Environment: See run_manifest.json
For Technical Validation
Processing summary: processing_summary.csv
Quality reports: dataset/validation/I3D/quality_reports/
Statistical analyses: dataset/validation/I3D/statistical_analyses/
Decision log: decision_log.jsonl
🔧 Configuration Options
Force Re-transcription
text:
  force_retranscribe: true  # Ignore transcript cache
Deterministic STT Mode
speech_recognition:
  deterministic_mode: true  # Disable dynamic_energy_threshold
Disable Caching
text:
  use_transcript_cache: false
  use_translation_cache: false
📖 Citation
{env_info.get('citation', '[Your BibTeX Citation]')}
📧 Contact
See paper for contact information and data availability details.

🏷️ Version Info
Pipeline Version: {VERSION}
Pipeline Name: {PIPELINE_NAME}
Report Generated: {datetime.now().isoformat()}
Python: {env_info['python_version'].split()[0]}
Platform: {env_info['platform']['system']} {env_info['platform']['release']}
Generated by {PIPELINE_NAME} v{VERSION} - Production-Ready for Scientific Data
"""
    report_path = paths.paths['processed'] / 'I3D_EXTRACTION_REPORT.md'
    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(markdown_content)

    return report_path

# ==================== MAIN PROCESSOR ====================
class I3DProcessor:
    """Main processor for I3D Dataset - Scientific Data Ready"""
    
    def __init__(self, config: I3DConfig, paths: I3DPathManager, suffix=''):  
        self.suffix = suffix  
        self.config = config
        self.paths = paths
        checkpoint_dir = paths.paths['checkpoints'] / f'checkpoint{suffix}' if suffix else paths.paths['checkpoints']
        checkpoint_dir.mkdir(parents=True, exist_ok=True)
        self.checkpoint_manager = CheckpointManager(checkpoint_dir)
        self.transcript_cache = TranscriptCacheManager(paths.paths['cache'])
        self.translation_cache = TranslationCacheManager(paths.paths['cache'])
        self.decision_log = DecisionLogManager(paths.paths['decision_logs'])
        self.env_info = capture_environment()
        self.status_by_file = {}  # {filename: {step: True/False}}   
    
    def _update_file_status(self, filename: str, step: str, success: bool):
        """Update status for a specific processing step"""
        if filename not in self.status_by_file:
            self.status_by_file[filename] = {
                'wav_conversion': False,
                'transcript_extraction': False,
                'translation': False,
                'audio_features': False,
                'pause_features': False,
                'landmarks': False,
                'completed': False
            }
        
        self.status_by_file[filename][step] = success
        
        # Check if all steps completed
        required_steps = ['wav_conversion', 'transcript_extraction', 'audio_features', 'landmarks']
        all_complete = all(self.status_by_file[filename].get(s, False) for s in required_steps)
        
        # Translation is optional (can fail but file still considered complete)
        self.status_by_file[filename]['completed'] = all_complete

    def _get_file_status(self, filename: str, step: str) -> bool:
        """Check if a specific step was completed for a file"""
        return self.status_by_file.get(filename, {}).get(step, False)

    def _is_file_completed(self, filename: str) -> bool:
        """Check if all required steps are completed for a file"""
        return self.status_by_file.get(filename, {}).get('completed', False)

    def _convert_to_wav(self, video_path: Path, label_name: str, logger: I3DLogger) -> Optional[Path]:
        """Convert video to WAV"""
        if label_name == 'lie':
            output_dir = self.paths.paths['audio_wav_lie']
        else:
            output_dir = self.paths.paths['audio_wav_truth']
        
        wav_path = output_dir / f"{video_path.stem}.wav"
        
        if wav_path.exists():
            logger.log(f"WAV already exists: {wav_path.name}")
            return wav_path
        
        try:
            subprocess.run([
                "ffmpeg", "-i", str(video_path),
                "-vn",
                "-acodec", "pcm_s16le",
                "-ar", "16000",
                "-ac", "1",
                "-af", "highpass=f=80,lowpass=f=3000,volume=3.0,afftdn=nf=-25,equalizer=f=1000:width_type=h:width=200:g=3",
                "-y", str(wav_path)
            ], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            
            logger.log(f"✓ Converted to WAV: {video_path.name}")
            return wav_path
        
        except subprocess.CalledProcessError as e:
            logger.log_error(f"Error converting {video_path}: {e}")
            return None
        except FileNotFoundError:
            logger.log_error("ffmpeg not found. Please install ffmpeg.")
            return None
    
    def _save_datasets(self, data: List[Dict], logger: I3DLogger) -> Dict:
        """Save all datasets"""
        if not data:
            logger.log_warning("⚠️ No data to save")
            return {}
        
        df = pd.DataFrame(data)
        output_paths = {}
        
        # ✅ TAMBAHKAN SUFFIX KE SEMUA PATH
        text_cols = ['filename', 'text_indonesian_original', 'text_indonesian_normalized', 
                    'label', 'dataset', 'transcript_source', 'transcript_from_cache',
                    'transcript_attempt', 'transcript_mode', 'stt_energy_threshold',
                    'char_count_id', 'word_count_id',
                    'sentiment_id', 'subjectivity_id', 'complexity_id', 'lexical_diversity_id']
        df_text_id = df[text_cols].copy()
        
        text_id_path = self.paths.paths['text'] / f'TextDataset_Indonesian{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_text_id.to_csv(text_id_path, index=False, encoding='utf-8')
        output_paths['text_indonesian'] = str(text_id_path)
        logger.log(f"✓ Text (Indonesian) saved: {text_id_path}")
        
        # DATASET 2: Text (English)
        text_en_cols = ['filename', 'text_english', 'label', 'dataset', 'transcript_source',
                        'char_count_en', 'word_count_en',
                        'sentiment_en', 'subjectivity_en', 'complexity_en', 'lexical_diversity_en']
        df_text_en = df[text_en_cols].copy()
        
        text_en_path = self.paths.paths['text'] / f'TextDataset_English{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_text_en.to_csv(text_en_path, index=False, encoding='utf-8')
        output_paths['text_english'] = str(text_en_path)
        logger.log(f"✓ Text (English) saved: {text_en_path}")
        
        # DATASET 3: Number Features
        number_cols = ['filename', 'label', 'dataset', 'has_numbers', 'number_count',
                    'has_vague_quantifiers', 'has_exact_quantifiers', 'number_word_ratio',
                    'has_large_numbers', 'has_decimal_numbers']
        df_numbers = df[number_cols].copy()
        
        numbers_path = self.paths.paths['text'] / f'NumberFeatures{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_numbers.to_csv(numbers_path, index=False, encoding='utf-8')
        output_paths['number_features'] = str(numbers_path)
        logger.log(f"✓ Number features saved: {numbers_path}")
        
        # DATASET 4: Audio Features (94)
        audio_feature_names = create_audio_feature_names()
        audio_cols = ['filename', 'label', 'dataset'] + audio_feature_names + [
            'audio_quality_score', 'audio_duration', 'audio_snr', 'audio_snr_computation',
            'audio_rms', 'audio_spectral_centroid', 'audio_dynamic_range'
        ]
        df_audio = df[audio_cols].copy()
        
        audio_path = self.paths.paths['audio'] / f'AudioDataset_Features{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_audio.to_csv(audio_path, index=False, encoding='utf-8')
        output_paths['audio_features'] = str(audio_path)
        logger.log(f"✓ Audio features (94) saved: {audio_path}")
        
        # DATASET 5: Pause Features
        pause_cols = ['filename', 'label', 'dataset'] + [col for col in df.columns if col.startswith('pause_')]
        df_pause = df[pause_cols].copy()
        
        pause_path = self.paths.paths['audio'] / f'PauseFeatures{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_pause.to_csv(pause_path, index=False, encoding='utf-8')
        output_paths['pause_features'] = str(pause_path)
        logger.log(f"✓ Pause features saved: {pause_path}")
        
        # DATASET 6: Multimodal (Full)
        multimodal_path = self.paths.paths['multimodal'] / f'MultimodalDataset_Full{self.suffix}.csv'  # ✅ TAMBAH suffix
        df.to_csv(multimodal_path, index=False, encoding='utf-8')
        output_paths['multimodal_full'] = str(multimodal_path)
        logger.log(f"✓ Multimodal (Full) saved: {multimodal_path}")
        
        # DATASET 7: Publication-Ready
        publication_cols = [
            'filename', 'label', 'dataset', 'transcript_source', 'transcript_from_cache',
            'reextraction_triggered', 'reextraction_success',
            'text_indonesian_normalized', 'text_english',
            'word_count_id', 'sentiment_en', 'lexical_diversity_id',
            'has_vague_quantifiers', 'has_exact_quantifiers', 'number_count',
            'mfcc1_mean', 'mfcc2_mean', 'mfcc3_mean', 'spectral_centroid_mean',
            'audio_quality_score', 'audio_snr',
            'pause_hesitation_score', 'pause_pause_frequency', 'pause_pause_ratio'
        ]
        publication_cols = [col for col in publication_cols if col in df.columns]
        df_publication = df[publication_cols].copy()
        
        publication_path = self.paths.paths['multimodal'] / f'PublicationDataset{self.suffix}.csv'  # ✅ TAMBAH suffix
        df_publication.to_csv(publication_path, index=False, encoding='utf-8')
        output_paths['publication'] = str(publication_path)
        logger.log(f"✓ Publication dataset saved: {publication_path}")
        
        # DATASET 8: Landmark
        landmark_path = self.paths.paths['visual'] / f'LandmarkDataset{self.suffix}.csv'  # ✅ TAMBAH suffix
        output_paths['landmark'] = str(landmark_path)
        logger.log(f"✓ Landmarks saved: {landmark_path}")
        
        return output_paths

    def _generate_statistics(self, data: List[Dict], stats: Dict, logger: I3DLogger):
        """Generate statistics and visualizations"""
        logger.log("\n📊 Generating statistics...")
        
        # Save processing statistics
        stats_path = self.paths.paths['statistical'] / 'processing_statistics.json'
        with open(stats_path, 'w', encoding='utf-8') as f:
            json.dump(convert_numpy_types(stats), f, indent=2)
        logger.log(f"✓ Statistics saved: {stats_path}")
        
        # Generate visualizations
        if self.config.get('output.generate_figures', True):
            self._generate_visualizations(data, stats, logger)
    
    def _generate_visualizations(self, data: List[Dict], stats: Dict, logger: I3DLogger):
        """Generate visualizations"""
        if not data:
            return
            
        df = pd.DataFrame(data)
        figures_dir = self.paths.paths['exploratory_figs']
        dpi = self.config.get('output.figure_dpi', 300)
        
        sns.set_style("whitegrid")
        
        # Visualization 1: Class Distribution
        plt.figure(figsize=(10, 6))
        label_counts = df['label'].value_counts()
        labels = ['TRUTH (0)', 'LIE (1)']
        colors = ['#3498db', '#e74c3c']
        
        plt.bar(labels, [label_counts.get(0, 0), label_counts.get(1, 0)], 
                color=colors, alpha=0.8, edgecolor='black')
        plt.title('Class Distribution - I3D Dataset', fontsize=16, fontweight='bold')
        plt.ylabel('Count', fontsize=12)
        plt.grid(axis='y', alpha=0.3)
        
        for i, v in enumerate([label_counts.get(0, 0), label_counts.get(1, 0)]):
            plt.text(i, v + 0.5, str(v), ha='center', fontweight='bold', fontsize=12)
        
        plt.tight_layout()
        plt.savefig(figures_dir / 'class_distribution.png', dpi=dpi, bbox_inches='tight')
        plt.close()
        
        # Visualization 2: Transcript Sources
        plt.figure(figsize=(10, 6))
        source_counts = df['transcript_source'].value_counts()
        plt.bar(source_counts.index, source_counts.values, color='skyblue', edgecolor='black')
        plt.title('Transcript Sources Distribution', fontsize=16, fontweight='bold')
        plt.xlabel('Source', fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.xticks(rotation=45, ha='right')
        plt.grid(axis='y', alpha=0.3)
        plt.tight_layout()
        plt.savefig(figures_dir / 'transcript_sources.png', dpi=dpi, bbox_inches='tight')
        plt.close()
        
        logger.log(f"✓ Visualizations saved: {figures_dir}")

    def process(self, checkpoint_data: Optional[Dict] = None):
        """Process I3D dataset with full reproducibility and smart resume"""
        
        # Initialize logger
        log_path, timestamp = self.paths.get_log_path()
        logger = I3DLogger(log_path)
        
        logger.log("="*70)
        logger.log(f"🎯 {PIPELINE_NAME} v{VERSION}")
        logger.log("="*70)
        logger.log("✨ Scientific Data Ready Features:")
        logger.log("   • Fixed 94 audio features (verified)")
        logger.log("   • MediaPipe: 478 face (468+10 iris) + 33 pose")
        logger.log("   • Runtime validation for landmark counts")
        logger.log("   • Explicit librosa parameters")
        logger.log("   • Deterministic mode available")
        logger.log("   • Complete data dictionary")
        logger.log("   • Run manifest for reproducibility")
        logger.log("   • Per-file processing summary")
        logger.log("   • Decision logging (JSONL)")
        logger.log("   • Environment capture")
        logger.log("   • Ethics & privacy notes")
        logger.log("   • ✅ Smart resume (retry failed steps only)")
        logger.log("="*70)
        
        # Log environment
        logger.log(f"\n🖥️ Environment:")
        logger.log(f"   Python: {self.env_info['python_version'].split()[0]}")
        logger.log(f"   Platform: {self.env_info['platform']['system']} {self.env_info['platform']['release']}")
        logger.log(f"   NumPy: {self.env_info['packages'].get('numpy', 'N/A')}")
        logger.log(f"   Librosa: {self.env_info['packages'].get('librosa', 'N/A')}")
        logger.log(f"   MediaPipe: {self.env_info['packages'].get('mediapipe', 'N/A')}")
        
        # Check source folders
        if not self.paths.paths['source_lie'].exists():
            raise FileNotFoundError(f"LIE folder not found: {self.paths.paths['source_lie']}")
        if not self.paths.paths['source_truth'].exists():
            raise FileNotFoundError(f"TRUTH folder not found: {self.paths.paths['source_truth']}")
        
        # Get video files
        lie_files = get_valid_video_files(self.paths.paths['source_lie'])
        truth_files = get_valid_video_files(self.paths.paths['source_truth'])
        total_files = len(lie_files) + len(truth_files)
        all_video_files = lie_files + truth_files
        
        logger.log(f"\n📁 Files found:")
        logger.log(f"   - LIE   : {len(lie_files)} files")
        logger.log(f"   - TRUTH : {len(truth_files)} files")
        logger.log(f"   - TOTAL : {total_files} files")
        
        # Count manual transcripts and cached transcripts
        manual_available = sum(1 for f in all_video_files if f in MANUAL_TRANSCRIPTS)
        cached_available = sum(1 for f in all_video_files if self.transcript_cache.exists(f))
        
        logger.log(f"\n📝 Transcript sources:")
        logger.log(f"   - Manual transcripts : {manual_available}/{total_files}")
        logger.log(f"   - Cached transcripts : {cached_available}/{total_files}")
        logger.log(f"   - Translation cache  : {self.translation_cache.get_cache_size()} entries")
        
        force_retranscribe = self.config.get('text.force_retranscribe', False)
        deterministic_mode = self.config.get('speech_recognition.deterministic_mode', False)
        
        if force_retranscribe:
            logger.log(f"\n⚠️  FORCE RETRANSCRIBE enabled - ignoring transcript cache")
        if deterministic_mode:
            logger.log(f"\n🔒 DETERMINISTIC MODE enabled - dynamic_energy_threshold=False")
        
        if total_files == 0:
            raise ValueError("No video files found!")
        
        # Initialize or restore state
        if checkpoint_data:
            stats = checkpoint_data.get('stats', {})
            text_audio_data = checkpoint_data.get('text_audio_data', [])
            failed_samples = checkpoint_data.get('failed_samples', [])
            processed_files = set(checkpoint_data.get('processed_files', []))
            
            # ✅ NEW: Restore status tracking
            self.status_by_file = checkpoint_data.get('status_by_file', {})
            
            logger.log(f"✓ Resumed: {len(processed_files)} files already processed")
            
            # ✅ NEW: Count partial vs complete
            complete_count = sum(1 for s in self.status_by_file.values() if s.get('completed', False))
            partial_count = len(processed_files) - complete_count
            
            if partial_count > 0:
                logger.log(f"  └─ {complete_count} fully completed, {partial_count} partial (will retry failed steps)")
        else:
            # Initialize statistics
            stats = {
                'total_files': total_files,
                'successful_text_conversions': 0,
                'failed_text_conversions': 0,
                'successful_audio_extractions': 0,
                'failed_audio_extractions': 0,
                'successful_translations': 0,
                'failed_translations': 0,
                'successful_landmark_extractions': 0,
                'failed_landmark_extractions': 0,
                'texts_with_numbers': 0,
                'numbers_normalized': 0,
                'total_frames_processed': 0,
                'face_detected': 0,
                'iris_detected': 0,
                'pose_detected': 0,
                'pause_features_extracted': 0,
                'reextraction_triggered': 0,
                'reextraction_successful': 0,
                'manual_transcripts_used': 0,
                'cached_transcripts_used': 0,
                'cached_translations_used': 0,
                'landmark_count_warnings': 0,
                'errors': []
            }
            text_audio_data = []
            failed_samples = []
            processed_files = set()
            self.status_by_file = {}  # ✅ NEW
        
        # Initialize processors
        preprocessor = VideoPreprocessor(self.config)
        audio_processor = AudioProcessor(self.config)
        landmark_extractor = LandmarkExtractor(self.config, preprocessor)
        
        # MediaPipe configuration
        face_mesh_config = self.config.get('mediapipe.face_mesh', {})
        pose_config = self.config.get('mediapipe.pose', {})
        
        # CSV for landmarks
        landmark_csv_path = self.paths.paths['visual'] / f'LandmarkDataset{self.suffix}.csv' 
        
        # Check if we're resuming and CSV already exists
        if checkpoint_data and landmark_csv_path.exists():
            landmark_csv = open(landmark_csv_path, 'a', newline='', encoding='utf-8')
            landmark_writer = csv.writer(landmark_csv)
        else:
            landmark_csv = open(landmark_csv_path, 'w', newline='', encoding='utf-8')
            landmark_writer = csv.writer(landmark_csv)
            
            # Landmark header
            header = ['Video_Name', 'Frame']
            for i in range(478):  # 468 face + 10 iris
                header.extend([f'Landmark_{i}_X', f'Landmark_{i}_Y', f'Landmark_{i}_Z'])
            for i in range(33):  # 33 pose
                header.extend([f'Pose_{i}_X', f'Pose_{i}_Y', f'Pose_{i}_Z'])
            header.append('Class')
            landmark_writer.writerow(header)
        
        # Process each label folder
        start_time = datetime.now()
        files_processed = len(processed_files)
        
        try:
            for label_name, folder in [('lie', self.paths.paths['source_lie'])]:
                files = get_valid_video_files(folder)
                
                logger.log(f"\n📁 Processing {len(files)} files from {label_name}")
                
                # Open MediaPipe context
                with mp_face_mesh.FaceMesh(**face_mesh_config) as face_mesh, \
                    mp_pose.Pose(**pose_config) as pose:
                    
                    # ========================================
                    # ✅ SMART RESUME LOOP (UPDATED)
                    # ========================================
                    
                    # Process each file
                    for filename in tqdm(files, desc=f"Processing {label_name}", unit="file"):
                        # ✅ SMART RESUME: Check completion status
                        if filename in processed_files:
                            if self._is_file_completed(filename):
                                logger.log(f"⏭️  Skipping (fully completed): {filename}")
                                continue
                            else:
                                logger.log(f"🔄 Resuming (partial completion): {filename}")
                                status = self.status_by_file.get(filename, {})
                                logger.log(f"  └─ Previous status: {status}")
                        
                        # Per-file flags
                        file_reextraction_triggered = False
                        file_reextraction_success = False
                        reextraction_params = None
                        
                        try:
                            video_path = folder / filename
                            video_name = video_path.stem
                            
                            if filename not in processed_files:
                                logger.log(f"\n🎬 Processing: {filename}")
                            
                            # Get label
                            label = 1 if label_name == 'lie' else 0
                            
                            # ========================================
                            # STEP 1: Convert to WAV
                            # ========================================
                            wav_path = None
                            
                            if self._get_file_status(filename, 'wav_conversion'):
                                logger.log(f"  ✓ WAV conversion: SKIP (already done)")
                                # Reconstruct WAV path
                                if label_name == 'lie':
                                    wav_path = self.paths.paths['audio_wav_lie'] / f"{video_name}.wav"
                                else:
                                    wav_path = self.paths.paths['audio_wav_truth'] / f"{video_name}.wav"
                                
                                if not wav_path.exists():
                                    logger.log_warning(f"  ⚠️ WAV file missing, will retry")
                                    self._update_file_status(filename, 'wav_conversion', False)
                                    wav_path = None
                            
                            if wav_path is None:
                                logger.log(f"  🔄 WAV conversion: PROCESSING")
                                wav_path = self._convert_to_wav(video_path, label_name, logger)
                                
                                if wav_path and wav_path.exists():
                                    self._update_file_status(filename, 'wav_conversion', True)
                                    logger.log(f"  ✓ WAV conversion: SUCCESS")
                                else:
                                    self._update_file_status(filename, 'wav_conversion', False)
                                    logger.log_error(f"  ❌ WAV conversion: FAILED")
                                    stats['failed_text_conversions'] += 1
                                    stats['failed_audio_extractions'] += 1
                                    failed_samples.append({'filename': filename, 'reason': 'WAV conversion failed', 'label': label})
                                    processed_files.add(filename)
                                    continue
                            
                            # ========================================
                            # STEP 2: Extract text
                            # ========================================
                            text = None
                            text_success = False
                            text_metadata = {}
                            
                            if self._get_file_status(filename, 'transcript_extraction'):
                                logger.log(f"  ✓ Transcript: SKIP (already done)")
                                # Load from cache
                                cached = self.transcript_cache.load_transcript(filename)
                                if cached:
                                    text = cached['text']
                                    text_success = True
                                    text_metadata = {
                                        'source': cached['source'],
                                        'from_cache': True,
                                        'attempt': cached.get('attempt', 0),
                                        'mode': cached.get('mode', 'normal'),
                                        'stt_energy_threshold': cached.get('stt_energy_threshold', 0.0)
                                    }
                                    logger.log(f"  └─ ✓ Loaded from cache: '{text[:50]}...'")
                                else:
                                    logger.log_warning(f"  ⚠️ Transcript cache missing, will retry")
                                    self._update_file_status(filename, 'transcript_extraction', False)
                            
                            if not text_success:
                                logger.log(f"  🔄 Transcript: PROCESSING")
                                text, text_success, text_metadata = audio_processor.extract_text(
                                    wav_path, logger, 
                                    ultra_mode=False, 
                                    filename=filename,
                                    transcript_cache=self.transcript_cache,
                                    decision_log=self.decision_log,
                                    force_retranscribe=force_retranscribe
                                )
                                
                                if text_success:
                                    self._update_file_status(filename, 'transcript_extraction', True)
                                    logger.log(f"  ✓ Transcript: SUCCESS")
                                    
                                    if text_metadata['from_cache']:
                                        stats['cached_transcripts_used'] += 1
                                    if text_metadata['source'] == 'manual':
                                        stats['manual_transcripts_used'] += 1
                                else:
                                    self._update_file_status(filename, 'transcript_extraction', False)
                                    logger.log_warning(f"  ⚠️ Transcript: FAILED (will try re-extraction)")
                            
                            # STEP 3: Re-extraction if failed
                            if not text_success and self.config.get('reextraction.enabled', True):
                                logger.log(f"  └─ 🔄 Triggering re-extraction (ultra mode)...")
                                stats['reextraction_triggered'] += 1
                                file_reextraction_triggered = True
                                
                                # Get audio dBFS for decision logging
                                try:
                                    audio = AudioSegment.from_wav(wav_path)
                                    audio_dbfs = audio.dBFS
                                except:
                                    audio_dbfs = 0.0
                                
                                # Enhance audio (ultra mode) with fixed parameters
                                enhanced_path, enhancement_params = audio_processor.enhance_audio(
                                    wav_path, logger, ultra_mode=True
                                )
                                
                                reextraction_params = enhancement_params
                                
                                # Extract text (ultra mode)
                                text, text_success, text_metadata = audio_processor.extract_text(
                                    enhanced_path, logger, 
                                    ultra_mode=True, 
                                    filename=filename,
                                    transcript_cache=self.transcript_cache,
                                    decision_log=self.decision_log,
                                    force_retranscribe=False
                                )
                                
                                # Log re-extraction decision
                                if self.config.get('reextraction.log_decisions', True):
                                    self.decision_log.log_reextraction(
                                        filename=filename,
                                        reason='text_extraction_failed',
                                        audio_dbfs=audio_dbfs,
                                        params=enhancement_params,
                                        success=text_success
                                    )
                                
                                if text_success:
                                    self._update_file_status(filename, 'transcript_extraction', True)
                                    stats['reextraction_successful'] += 1
                                    file_reextraction_success = True
                                    logger.log(f"  └─ ✅ Re-extraction successful!")
                            
                            # Final transcript status
                            if text_success:
                                stats['successful_text_conversions'] += 1
                            else:
                                stats['failed_text_conversions'] += 1
                                failed_samples.append({
                                    'filename': filename, 
                                    'reason': 'Text extraction failed', 
                                    'label': label
                                })
                                text = ""
                            
                            # ========================================
                            # STEP 4: Extract number features
                            # ========================================
                            number_features = extract_number_features(text)
                            if number_features['has_numbers']:
                                stats['texts_with_numbers'] += 1
                                logger.log(f"  └─ 🔢 Numbers detected: {number_features['number_count']}")
                            
                            # ========================================
                            # STEP 5: Normalize numbers
                            # ========================================
                            text_normalized = ""
                            if self.config.get('text.normalize_numbers', True) and text:
                                text_normalized = normalize_numbers_in_text(text)
                                if text != text_normalized:
                                    stats['numbers_normalized'] += 1
                                    logger.log(f"  └─ ✓ Numbers normalized")
                            else:
                                text_normalized = text
                            
                            # ========================================
                            # STEP 6: Translate to English
                            # ========================================
                            text_en = ""
                            translation_success = False
                            
                            if self._get_file_status(filename, 'translation'):
                                logger.log(f"  ✓ Translation: SKIP (already done)")
                                # Try to load from cache
                                cached_translation = self.translation_cache.get_translation(text_normalized)
                                if cached_translation:
                                    text_en = cached_translation
                                    translation_success = True
                                    stats['cached_translations_used'] += 1
                                    logger.log(f"  └─ ✓ Loaded from cache")
                                else:
                                    logger.log_warning(f"  ⚠️ Translation cache missing, will retry")
                                    self._update_file_status(filename, 'translation', False)
                            
                            if not translation_success and text_success and self.config.get('text.translate_to_english', True):
                                logger.log(f"  🔄 Translation: PROCESSING")
                                
                                text_en = translate_text_with_cache(
                                    text_normalized, 
                                    self.translation_cache,
                                    self.decision_log,
                                    filename,
                                    logger=logger
                                )
                                
                                if text_en:
                                    self._update_file_status(filename, 'translation', True)
                                    stats['successful_translations'] += 1
                                    logger.log(f"  ✓ Translation: SUCCESS")
                                else:
                                    self._update_file_status(filename, 'translation', False)
                                    stats['failed_translations'] += 1
                                    logger.log_warning(f"  ⚠️ Translation: FAILED")
                                    text_en = ""
                            
                            # ========================================
                            # STEP 7: Analyze texts
                            # ========================================
                            features_id = analyze_text_content(text_normalized, 'id')
                            features_en = analyze_text_content(text_en, 'en')
                            
                            # ========================================
                            # STEP 8: Extract audio features (94)
                            # ========================================
                            audio_features = None
                            
                            if self._get_file_status(filename, 'audio_features'):
                                logger.log(f"  ✓ Audio features: SKIP (already done)")
                                # Try to find in text_audio_data
                                existing_entry = next((e for e in text_audio_data if e['filename'] == filename), None)
                                if existing_entry:
                                    # Extract audio features from existing entry
                                    audio_feature_names = create_audio_feature_names()
                                    audio_features = np.array([existing_entry[name] for name in audio_feature_names])
                                    logger.log(f"  └─ ✓ Loaded from checkpoint data")
                                else:
                                    logger.log_warning(f"  ⚠️ Audio features data missing, will retry")
                                    self._update_file_status(filename, 'audio_features', False)
                            
                            if audio_features is None:
                                logger.log(f"  🔄 Audio features: PROCESSING")
                                audio_features = extract_audio_features(wav_path, self.config, logger)
                                
                                if audio_features is not None:
                                    self._update_file_status(filename, 'audio_features', True)
                                    stats['successful_audio_extractions'] += 1
                                    logger.log(f"  ✓ Audio features: SUCCESS")
                                else:
                                    self._update_file_status(filename, 'audio_features', False)
                                    stats['failed_audio_extractions'] += 1
                                    logger.log_warning(f"  ⚠️ Audio features: FAILED")
                                    audio_features = np.zeros(94)
                            
                            # ========================================
                            # STEP 9: Extract pause features
                            # ========================================
                            pause_features = extract_pause_silence_features(wav_path, self.config, logger)
                            if pause_features['num_pauses'] > 0:
                                stats['pause_features_extracted'] += 1
                            
                            # ========================================
                            # STEP 10: Check audio quality
                            # ========================================
                            quality_info = check_audio_quality(wav_path, self.config, logger)
                            
                            # ========================================
                            # STEP 11: Store TEXT + AUDIO data
                            # ========================================
                            # Remove old entry if exists (for partial resume)
                            text_audio_data = [e for e in text_audio_data if e['filename'] != filename]
                            
                            data_entry = {
                                'filename': filename,
                                'text_indonesian_original': text,
                                'text_indonesian_normalized': text_normalized,
                                'text_english': text_en,
                                'label': label,
                                'dataset': 'I3D',
                                
                                # Transcript metadata
                                'transcript_source': text_metadata.get('source', 'unknown'),
                                'transcript_from_cache': text_metadata.get('from_cache', False),
                                'transcript_attempt': text_metadata.get('attempt', 0),
                                'transcript_mode': text_metadata.get('mode', 'normal'),
                                'stt_energy_threshold': text_metadata.get('stt_energy_threshold', 0.0),
                                
                                # Re-extraction metadata
                                'reextraction_triggered': file_reextraction_triggered,
                                'reextraction_success': file_reextraction_success,
                                'reextraction_params': json.dumps(reextraction_params) if reextraction_params else '',
                                
                                # Number features
                                'has_numbers': number_features['has_numbers'],
                                'number_count': number_features['number_count'],
                                'has_vague_quantifiers': number_features['has_vague_quantifiers'],
                                'has_exact_quantifiers': number_features['has_exact_quantifiers'],
                                'number_word_ratio': number_features['number_word_ratio'],
                                'has_large_numbers': number_features['has_large_numbers'],
                                'has_decimal_numbers': number_features['has_decimal_numbers'],
                                
                                # Indonesian text features
                                'char_count_id': features_id['char_count'],
                                'word_count_id': features_id['word_count'],
                                'sentiment_id': features_id['sentiment'],
                                'subjectivity_id': features_id['subjectivity'],
                                'complexity_id': features_id['complexity'],
                                'lexical_diversity_id': features_id['lexical_diversity'],
                                
                                # English text features
                                'char_count_en': features_en['char_count'],
                                'word_count_en': features_en['word_count'],
                                'sentiment_en': features_en['sentiment'],
                                'subjectivity_en': features_en['subjectivity'],
                                'complexity_en': features_en['complexity'],
                                'lexical_diversity_en': features_en['lexical_diversity'],
                                
                                # Audio quality
                                'audio_quality_score': quality_info.get('quality_score', 0),
                                'audio_duration': quality_info.get('duration', 0),
                                'audio_snr': quality_info.get('snr', 0),
                                'audio_snr_computation': quality_info.get('snr_computation', ''),
                                'audio_rms': quality_info.get('rms', 0),
                                'audio_spectral_centroid': quality_info.get('spectral_centroid', 0),
                                'audio_dynamic_range': quality_info.get('dynamic_range', 0)
                            }
                            
                            # Add audio features (94)
                            audio_feature_names = create_audio_feature_names()
                            for i, feature_name in enumerate(audio_feature_names):
                                data_entry[feature_name] = float(audio_features[i])
                            
                            # Add pause features
                            for feature_name, feature_value in pause_features.items():
                                data_entry[f'pause_{feature_name}'] = feature_value
                            
                            text_audio_data.append(data_entry)
                            
                            # ========================================
                            # STEP 12: Extract LANDMARKS
                            # ========================================
                            landmarks_done = False
                            
                            if self._get_file_status(filename, 'landmarks'):
                                logger.log(f"  ✓ Landmarks: SKIP (already done)")
                                # Check if landmarks exist in CSV
                                # (We can't easily verify this without reading the entire CSV,
                                #  so we trust the status flag)
                                landmarks_done = True
                                stats['successful_landmark_extractions'] += 1
                            
                            if not landmarks_done:
                                logger.log(f"  🔄 Landmarks: PROCESSING")
                                
                                cap = cv2.VideoCapture(str(video_path))
                                
                                if not cap.isOpened():
                                    logger.log_error(f"❌ Failed to open video: {filename}")
                                    self._update_file_status(filename, 'landmarks', False)
                                    stats['failed_landmark_extractions'] += 1
                                    processed_files.add(filename)
                                    continue
                                
                                frame_count = 0
                                landmark_detected_count = 0
                                sample_rate = self.config.get('video.sample_rate', 1)
                                
                                detection_summary = {
                                    'total_frames': 0,
                                    'face_detected': 0,
                                    'iris_detected': 0,
                                    'pose_detected': 0,
                                    'strategies_used': {}
                                }
                                
                                while cap.isOpened():
                                    ret, frame = cap.read()
                                    if not ret:
                                        break
                                    
                                    if frame_count % sample_rate != 0:
                                        frame_count += 1
                                        continue
                                    
                                    stats['total_frames_processed'] += 1
                                    detection_summary['total_frames'] += 1
                                    
                                    # Extract face landmarks
                                    face_landmarks_dict, iris_count, strategy = landmark_extractor.extract_with_multi_strategy(
                                        frame, face_mesh, iris_focus=True
                                    )
                                    
                                    # ✅ Validate face/iris landmark count
                                    if face_landmarks_dict:
                                        num_face_landmarks = len(face_landmarks_dict)
                                        expected_face_landmarks = 478  # 468 face + 10 iris
                                        
                                        if num_face_landmarks != expected_face_landmarks:
                                            logger.log_warning(
                                                f"⚠️ Unexpected face/iris landmark count: {num_face_landmarks} "
                                                f"(expected {expected_face_landmarks}) for {filename} frame {frame_count}. "
                                                f"MediaPipe version: {self.env_info['packages'].get('mediapipe', 'unknown')}"
                                            )
                                            stats['landmark_count_warnings'] += 1
                                        
                                        detection_summary['face_detected'] += 1
                                        stats['face_detected'] += 1
                                        
                                        if strategy:
                                            detection_summary['strategies_used'][strategy] = \
                                                detection_summary['strategies_used'].get(strategy, 0) + 1
                                    
                                    if iris_count >= 5:
                                        detection_summary['iris_detected'] += 1
                                        stats['iris_detected'] += 1
                                        landmark_detected_count += 1
                                    
                                    # Extract pose landmarks
                                    pose_landmarks_dict = None
                                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                                    pose_results = pose.process(frame_rgb)
                                    
                                    if pose_results.pose_landmarks:
                                        pose_landmarks_dict = {}
                                        for idx, landmark in enumerate(pose_results.pose_landmarks.landmark):
                                            pose_landmarks_dict[idx] = [landmark.x, landmark.y, landmark.z]
                                        
                                        # ✅ Validate pose landmark count
                                        num_pose_landmarks = len(pose_landmarks_dict)
                                        expected_pose_landmarks = 33
                                        
                                        if num_pose_landmarks != expected_pose_landmarks:
                                            logger.log_warning(
                                                f"⚠️ Unexpected pose landmark count: {num_pose_landmarks} "
                                                f"(expected {expected_pose_landmarks}) for {filename} frame {frame_count}. "
                                                f"MediaPipe version: {self.env_info['packages'].get('mediapipe', 'unknown')}"
                                            )
                                            stats['landmark_count_warnings'] += 1
                                        
                                        detection_summary['pose_detected'] += 1
                                        stats['pose_detected'] += 1
                                    
                                    # Write to CSV
                                    if face_landmarks_dict or pose_landmarks_dict:
                                        row = [video_name, frame_count]
                                        
                                        # Face + Iris landmarks (478)
                                        for i in range(478):
                                            if face_landmarks_dict and i in face_landmarks_dict:
                                                row.extend(face_landmarks_dict[i])
                                            else:
                                                row.extend([0.0, 0.0, 0.0])
                                        
                                        # Pose landmarks (33)
                                        for i in range(33):
                                            if pose_landmarks_dict and i in pose_landmarks_dict:
                                                row.extend(pose_landmarks_dict[i])
                                            else:
                                                row.extend([0.0, 0.0, 0.0])
                                        
                                        row.append(label)
                                        landmark_writer.writerow(row)
                                    
                                    frame_count += 1
                                
                                cap.release()
                                
                                # Log detection summary
                                if detection_summary['total_frames'] > 0:
                                    face_rate = (detection_summary['face_detected'] / detection_summary['total_frames']) * 100
                                    iris_rate = (detection_summary['iris_detected'] / detection_summary['total_frames']) * 100
                                    pose_rate = (detection_summary['pose_detected'] / detection_summary['total_frames']) * 100
                                    
                                    logger.log(f"  └─ ✓ Landmarks extracted: {landmark_detected_count} frames")
                                    logger.log(f"     Face: {face_rate:.1f}% ({detection_summary['face_detected']}/{detection_summary['total_frames']})")
                                    logger.log(f"     Iris: {iris_rate:.1f}% ({detection_summary['iris_detected']}/{detection_summary['total_frames']})")
                                    logger.log(f"     Pose: {pose_rate:.1f}% ({detection_summary['pose_detected']}/{detection_summary['total_frames']})")
                                    
                                    if landmark_detected_count > 0:
                                        self._update_file_status(filename, 'landmarks', True)
                                        stats['successful_landmark_extractions'] += 1
                                        logger.log(f"  ✓ Landmarks: SUCCESS")
                                    else:
                                        self._update_file_status(filename, 'landmarks', False)
                                        stats['failed_landmark_extractions'] += 1
                                        logger.log_warning(f"  ⚠️ Landmarks: FAILED")
                                else:
                                    self._update_file_status(filename, 'landmarks', False)
                                    stats['failed_landmark_extractions'] += 1
                            
                            # ========================================
                            # Mark as processed
                            # ========================================
                            processed_files.add(filename)
                            files_processed += 1
                            
                            # Log completion status
                            if self._is_file_completed(filename):
                                logger.log(f"  ✅ File FULLY COMPLETED: {filename}")
                            else:
                                logger.log(f"  ⚠️  File PARTIALLY COMPLETED: {filename}")
                                incomplete_steps = [
                                    step for step in ['wav_conversion', 'transcript_extraction', 'audio_features', 'landmarks']
                                    if not self._get_file_status(filename, step)
                                ]
                                logger.log(f"     Incomplete steps: {', '.join(incomplete_steps)}")
                            
                            # ========================================
                            # SAVE CHECKPOINT (with status tracking)
                            # ========================================
                            if self.config.get('checkpoint.enabled', True):
                                save_interval = self.config.get('checkpoint.save_interval', 5)
                                if files_processed % save_interval == 0:
                                    checkpoint_data_save = {
                                        'processed_files': list(processed_files),
                                        'text_audio_data': text_audio_data,
                                        'failed_samples': failed_samples,
                                        'stats': stats,
                                        'status_by_file': self.status_by_file  # ✅ NEW: Save status
                                    }
                                    self.checkpoint_manager.save_checkpoint(checkpoint_data_save)
                                    
                                    # Count completion stats
                                    complete_count = sum(1 for s in self.status_by_file.values() if s.get('completed', False))
                                    partial_count = len(processed_files) - complete_count
                                    
                                    logger.log(f"  └─ 💾 Checkpoint saved ({files_processed} files)")
                                    logger.log(f"     Complete: {complete_count}, Partial: {partial_count}")
                        
                        except Exception as e:
                            logger.log_error(f"❌ Error processing {filename}: {str(e)}")
                            import traceback
                            logger.log_error(traceback.format_exc())
                            
                            # Mark all steps as failed
                            self._update_file_status(filename, 'wav_conversion', False)
                            self._update_file_status(filename, 'transcript_extraction', False)
                            self._update_file_status(filename, 'translation', False)
                            self._update_file_status(filename, 'audio_features', False)
                            self._update_file_status(filename, 'landmarks', False)
                            
                            stats['errors'].append(filename)
                            stats['failed_text_conversions'] += 1
                            stats['failed_audio_extractions'] += 1
                            stats['failed_landmark_extractions'] += 1
                            failed_samples.append({'filename': filename, 'reason': str(e), 'label': label})
                            processed_files.add(filename)
                            continue
        
        finally:
            # Close landmark CSV
            landmark_csv.close()
        
        # Save datasets
        output_paths = self._save_datasets(text_audio_data, logger)
        
        # Save failed samples
        if failed_samples:
            failed_df = pd.DataFrame(failed_samples)
            failed_path = self.paths.paths['reextraction'] / 'failed_samples.csv'
            failed_df.to_csv(failed_path, index=False, encoding='utf-8')
            logger.log(f"\n⚠️ Failed samples saved: {failed_path}")
            logger.log(f"  └─ {len(failed_samples)} samples need manual review")
        
        # Generate statistics
        self._generate_statistics(text_audio_data, stats, logger)
        
        # Generate data dictionary
        end_time = datetime.now()
        if self.config.get('output.generate_data_dictionary', True):
            dict_path = generate_data_dictionary(self.paths)
            logger.log(f"\n✓ Data dictionary generated: {dict_path}")
        
        # Generate processing summary
        summary_path = generate_processing_summary(text_audio_data, self.paths)
        logger.log(f"✓ Processing summary generated: {summary_path}")
        
        # Generate run manifest
        if self.config.get('output.generate_run_manifest', True):
            manifest_path = generate_run_manifest(
                self.env_info, self.config, self.paths, stats, all_video_files
            )
            logger.log(f"✓ Run manifest generated: {manifest_path}")
        
        # Generate markdown report
        if self.config.get('output.generate_markdown_report', True):
            markdown_path = generate_markdown_report(
                stats, output_paths, self.paths, start_time, end_time, self.env_info, self.config
            )
            logger.log(f"✓ Markdown report generated: {markdown_path}")
        
        # ✅ Log landmark validation warnings
        if stats.get('landmark_count_warnings', 0) > 0:
            logger.log(f"\n⚠️ Landmark Count Warnings: {stats['landmark_count_warnings']}")
            logger.log(f"   Some frames had unexpected landmark counts.")
            logger.log(f"   This may indicate MediaPipe version differences.")
            logger.log(f"   See log for details.")
        
        # ✅ Log smart resume statistics
        logger.log(f"\n📊 Smart Resume Statistics:")
        complete_count = sum(1 for s in self.status_by_file.values() if s.get('completed', False))
        partial_count = len(processed_files) - complete_count
        logger.log(f"   - Fully completed files: {complete_count}")
        logger.log(f"   - Partially completed files: {partial_count}")
        
        if partial_count > 0:
            logger.log(f"\n💡 Partial files breakdown:")
            for filename, status in self.status_by_file.items():
                if not status.get('completed', False):
                    failed_steps = [step for step, success in status.items() 
                                if step != 'completed' and not success]
                    if failed_steps:
                        logger.log(f"   - {filename}: Failed steps: {', '.join(failed_steps)}")
        
        # Log cache statistics
        logger.log(f"\n📊 Cache Statistics:")
        logger.log(f"   - Transcript cache: {self.transcript_cache.transcripts_dir}")
        logger.log(f"   - Translation cache: {self.translation_cache.get_cache_size()} entries")
        logger.log(f"   - Decision log: {self.decision_log.decisions_file}")
        
        # Clear checkpoint after successful completion
        self.checkpoint_manager.clear_checkpoint()
        
        logger.finalize()
        
        return {
            'status': 'success',
            'stats': stats,
            'failed_samples': len(failed_samples),
            'output_paths': output_paths,
            'env_info': self.env_info,
            'smart_resume_stats': {
                'complete_count': complete_count,
                'partial_count': partial_count
            }
        }

# ==================== MAIN EXECUTION ====================
def main():
    """Main execution function"""
    
    print("="*70)
    print(f"🎯 {PIPELINE_NAME} v{VERSION}")
    print("="*70)
    print("✨ SCIENTIFIC DATA READY - Production Version")
    print()
    print("🔧 CRITICAL FIXES:")
    print("   ✅ Audio features: 94 (verified with assert)")
    print("   ✅ MediaPipe landmarks: 478 face (468+10 iris) + 33 pose")
    print("   ✅ Explicit librosa parameters (n_fft, hop_length, etc)")
    print("   ✅ Deterministic STT mode available")
    print("   ✅ Per-file re-extraction flags (not global)")
    print("   ✅ Complete data dictionary generated")
    print("   ✅ Run manifest for full reproducibility")
    print("   ✅ Processing summary (per-file quality metrics)")
    print("   ✅ Decision logging (JSONL format)")
    print("   ✅ Environment capture (Python, packages, FFmpeg)")
    print("   ✅ SNR computation documented (STFT parameters)")
    print("   ✅ Sentiment analysis caveat documented")
    print("   ✅ Filename leakage warning")
    print("   ✅ Missing landmark policy: (0.0, 0.0, 0.0)")
    print("   ✅ Ethics & privacy statement included")
    print()
    print("📦 Standard Features:")
    print("   • Language: Indonesian")
    print("   • 📝 Manual Transcript Support (33 files)")
    print("   • Transcript Freezing (cache after first extraction)")
    print("   • Translation Freezing (MD5-based cache)")
    print("   • Indonesian number normalization (0-Trillion)")
    print("   • Number features extraction (7 features)")
    print("   • Audio Features: 94 (MFCC + Spectral + Energy)")
    print("   • Pause/Silence Analysis (17 features)")
    print("   • FaceMesh Landmarks (468 facial + 10 iris = 478)")
    print("   • Pose Landmarks (33 body keypoints)")
    print("   • Advanced Video Preprocessing")
    print("   • Multi-level Zoom (up to 10x)")
    print("   • Integrated Re-extraction")
    print("   • 💾 AUTO-SAVE CHECKPOINT (Resume if interrupted)")
    print("   • Comprehensive Reporting")
    print("="*70)
    
    # Initialize configuration
    base_dir = os.getcwd()
    config_path = os.path.join(base_dir, "config_I3D.yaml")
    
    config = I3DConfig(config_path if os.path.exists(config_path) else None)
    
    if not os.path.exists(config_path):
        config.save_to_file(config_path)
        print(f"\n✓ Default configuration saved to: {config_path}")
        print("  You can edit this file to customize settings.")
        print()
        print("  Key options:")
        print("    text.force_retranscribe: false  # Set true to ignore transcript cache")
        print("    speech_recognition.deterministic_mode: true  # ✅ TRUE by default (reproducible)")
        print("    text.use_transcript_cache: true  # Enable/disable transcript caching")
        print("    text.use_translation_cache: true # Enable/disable translation caching")
    
    # ✅ NEW: Check deterministic mode and warn if disabled
    deterministic_mode = config.get('speech_recognition.deterministic_mode', True)
    if not deterministic_mode:
        print("\n" + "="*70)
        print("⚠️  WARNING: DETERMINISTIC MODE DISABLED")
        print("="*70)
        print("   • Dynamic energy threshold enabled (non-reproducible)")
        print("   • STT results may vary between runs")
        print("   • For Scientific Data submission, deterministic_mode=true is recommended")
        print("="*70)
        
        response = input("\nContinue with non-deterministic mode? (y/n): ")
        if response.lower() != 'y':
            print("❌ Processing cancelled.")
            print("💡 Set deterministic_mode: true in config_I3D.yaml")
            return
    else:
        print("\n✅ Deterministic mode ENABLED (reproducible STT)")

    # Initialize path manager
    path_manager = I3DPathManager(base_dir)
    path_manager.create_directories()
    
    # Initialize checkpoint manager
    checkpoint_manager = CheckpointManager(path_manager.paths['checkpoints'])
    
    # Check for existing checkpoint
    checkpoint_data = None
    if checkpoint_manager.checkpoint_exists():
        print("\n" + "="*70)
        print("🔄 CHECKPOINT DETECTED!")
        print("="*70)
        checkpoint_data_temp = checkpoint_manager.load_checkpoint()
        if checkpoint_data_temp:
            processed_count = len(checkpoint_data_temp.get('processed_files', []))
            print(f"   • Previously processed: {processed_count} files")
            print("="*70)
            
            response = input("\n🔄 Resume from checkpoint? (y/n): ")
            if response.lower() == 'y':
                print("✓ Resuming from checkpoint...")
                print("  └─ All caches will be preserved")
                checkpoint_data = checkpoint_data_temp
            else:
                response_clear = input("⚠️  Clear checkpoint and start fresh? (y/n): ")
                if response_clear.lower() == 'y':
                    checkpoint_manager.clear_checkpoint()
                    print("✓ Checkpoint cleared. Starting fresh...")
                    print("  └─ Note: Transcript and translation caches will still be used")
                    print("  └─ Set force_retranscribe=true in config to ignore transcript cache")
                else:
                    print("❌ Processing cancelled.")
                    return
    
    # Check cache status
    transcript_cache = TranscriptCacheManager(path_manager.paths['cache'])
    translation_cache = TranslationCacheManager(path_manager.paths['cache'])
    
    # Count existing caches
    transcript_count = len(list(transcript_cache.transcripts_dir.glob('*.json')))
    translation_count = translation_cache.get_cache_size()
    
    if transcript_count > 0 or translation_count > 0:
        print("\n" + "="*70)
        print("💾 CACHE DETECTED!")
        print("="*70)
        print(f"   • Transcript cache: {transcript_count} files")
        print(f"   • Translation cache: {translation_count} entries")
        print("="*70)
        print("✓ These caches will be used for deterministic processing")
        print("  └─ Transcripts will be loaded from cache (skip STT)")
        print("  └─ Translations will be loaded from cache (skip Google API)")
        print()
        
        force_retranscribe = config.get('text.force_retranscribe', False)
        if force_retranscribe:
            print("⚠️  WARNING: force_retranscribe=true in config")
            print("  └─ Transcript cache will be IGNORED")
            print("  └─ All files will be re-transcribed via STT")
        else:
            print("💡 TIP: To force re-transcription, set force_retranscribe=true in config")
    
    # Check ffmpeg
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, 
                      stderr=subprocess.PIPE, check=True)
        print("\n✓ ffmpeg detected")
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("\n❌ ffmpeg not found!")
        print("💡 Please install ffmpeg: https://ffmpeg.org/download.html")
        return
    
    # Confirm processing
    if not checkpoint_data:
        print("\n" + "="*70)
        print("⚠️  IMPORTANT FOR SCIENTIFIC DATA:")
        print("   • Internet required for translation (if not cached)")
        print("   • External APIs: Google STT + Google Translate")
        print("   • First run: Non-deterministic (API variability)")
        print("   • Subsequent runs: Deterministic for cached files")
        print("   • All outputs will be frozen and documented")
        print("   • 📝 33 manual transcripts will be used automatically")
        print("   • 🔒 Transcript caching enabled (deterministic)")
        print("   • 🌐 Translation caching enabled (deterministic)")
        print("   • 💾 Auto-checkpoint every 5 files")
        print("   • 📊 Complete data dictionary will be generated")
        print("   • 📋 Run manifest for reproducibility")
        print("   • ⚠️  Filename contains label (LIE_/TRUTH_) - remove before ML")
        print("   • 🔒 MediaPipe: 478 face (468+10 iris) + 33 pose landmarks")
        print("   • ⚠️  Missing landmarks: (0.0, 0.0, 0.0) - NOT actual position")
        print("   • 🛡️  Ethics: Interrogation data - see report for privacy notes")
        print("="*70)
        
        print("\n📁 Expected directory structure:")
        print("   dataset/raw/I3D/lie/       <- Place LIE videos here")
        print("   dataset/raw/I3D/truth/     <- Place TRUTH videos here")
        
        print("\n📂 Output structure:")
        print("   dataset/processed/I3D/     <- Processed datasets")
        print("   dataset/metadata/I3D/      <- Data dictionary + run manifest")
        print("   dataset/validation/I3D/    <- Quality reports + statistics")
        print("   dataset/_cache/I3D/        <- Transcript & translation cache")
        print("   dataset/_logs/I3D/         <- Processing logs & decisions")
        print("   dataset/_checkpoints/I3D/  <- Resume checkpoints")
        
        response = input("\n🚀 Start processing? (y/n): ")
        if response.lower() != 'y':
            print("❌ Processing cancelled.")
            return
    
    # Initialize processor
    processor = I3DProcessor(config, path_manager, suffix='_lie')
    
    # Process dataset
    try:
        result = processor.process(checkpoint_data)
        
        if result['status'] == 'success':
            print("\n" + "="*70)
            print("✅ I3D DATASET PROCESSING COMPLETE!")
            print("="*70)
            print(f"📊 Total files: {result['stats']['total_files']}")
            print(f"✓  Text extraction: {result['stats']['successful_text_conversions']}/{result['stats']['total_files']}")
            print(f"📝 Manual transcripts: {result['stats']['manual_transcripts_used']}")
            print(f"💾 Cached transcripts: {result['stats']['cached_transcripts_used']}")
            print(f"✓  Numbers normalized: {result['stats']['numbers_normalized']}/{result['stats']['texts_with_numbers']}")
            print(f"✓  Translation: {result['stats']['successful_translations']}/{result['stats']['successful_text_conversions']}")
            print(f"🌐 Cached translations: {result['stats']['cached_translations_used']}")
            print(f"✓  Audio features (94): {result['stats']['successful_audio_extractions']}/{result['stats']['total_files']}")
            print(f"✓  Landmark extraction: {result['stats']['successful_landmark_extractions']}/{result['stats']['total_files']}")
            print(f"👁️  Iris detection: {result['stats']['iris_detected']:,} frames")
            print(f"🔄 Re-extractions: {result['stats']['reextraction_triggered']} triggered, {result['stats']['reextraction_successful']} successful")
            print(f"⚠️  Failed samples: {result['failed_samples']}")
            print("="*70)
            print(f"\n📦 Output directory: {path_manager.paths['processed']}")
            print(f"📋 Detailed logs: {path_manager.paths['logs']}")
            print(f"💾 Cache directory: {path_manager.paths['cache']}")
            print(f"📊 Decision log: {path_manager.paths['decision_logs']}")
            print(f"📚 Metadata: {path_manager.paths['metadata']}")
            print("="*70)
            
            # Show cache statistics
            print("\n🔒 Deterministic Processing Summary:")
            print(f"   • Transcript cache: {result['stats']['cached_transcripts_used']}/{result['stats']['successful_text_conversions']} from cache")
            print(f"   • Translation cache: {result['stats']['cached_translations_used']}/{result['stats']['successful_translations']} from cache")
            print(f"   • Manual transcripts: {result['stats']['manual_transcripts_used']} files")
            print(f"   • Re-extractions: {result['stats']['reextraction_successful']}/{result['stats']['reextraction_triggered']} successful")
            
            cache_efficiency = 0
            if result['stats']['successful_text_conversions'] > 0:
                cache_efficiency = (result['stats']['cached_transcripts_used'] / result['stats']['successful_text_conversions']) * 100
            
            print(f"\n📈 Cache Efficiency: {cache_efficiency:.1f}%")
            if cache_efficiency > 80:
                print("   ✅ Excellent! Most transcripts loaded from cache")
            elif cache_efficiency > 50:
                print("   ✓  Good cache utilization")
            else:
                print("   💡 First run or cache cleared - building cache")
            
            print("\n💡 Next Run:")
            print("   • Transcripts will be loaded from cache (instant)")
            print("   • Translations will be loaded from cache (instant)")
            print("   • Processing will be much faster!")
            print("   • Results will be identical for cached files (deterministic)")
            
            print("\n🔧 Configuration Options:")
            print("   • Force re-transcription: set force_retranscribe=true in config")
            print("   • Deterministic STT: set deterministic_mode=true in config")
            print("   • Disable caching: set use_transcript_cache=false")
            
            print("\n📚 Documentation for Scientific Data:")
            print("   • Extraction report: dataset/processed/I3D/I3D_EXTRACTION_REPORT.md")
            print("   • Data dictionary: dataset/metadata/I3D/data_dictionary.csv")
            print("   • Run manifest: dataset/metadata/I3D/run_manifest.json")
            print("   • Processing summary: dataset/validation/I3D/quality_reports/processing_summary.csv")
            print("   • Decision log: dataset/_logs/I3D/decisions/decision_log.jsonl")
            print("   • Statistics: dataset/validation/I3D/statistical_analyses/processing_statistics.json")
            
            print("\n⚠️  IMPORTANT NOTES:")
            print("   • Audio features: 94 (verified with assert)")
            print("   • MediaPipe landmarks: 478 face (468+10 iris) + 33 pose")
            print("   • Filename contains label: Remove before ML training")
            print("   • Sentiment (Indonesian): Proxy only - use English version")
            print("   • Missing landmarks: (0.0, 0.0, 0.0) - NOT actual position")
            print("   • External APIs: Google STT + Translate (non-deterministic on first run)")
            print("   • Ethics: Interrogation data - consult IRB before public release")
            
            print("="*70 + "\n")
        else:
            print("\n❌ Processing failed!")
            
    except KeyboardInterrupt:
        print("\n\n⚠️  PROCESSING INTERRUPTED!")
        print("="*70)
        print("💾 Checkpoint has been saved automatically.")
        print("   Run the script again and choose 'Resume from checkpoint'")
        print("   to continue where you left off.")
        print()
        print("🔒 Cache Status:")
        print("   • Transcript cache: PRESERVED")
        print("   • Translation cache: PRESERVED")
        print("   • All processed transcripts are cached")
        print("   • Resume will use cached data (deterministic)")
        print("="*70)
    except Exception as e:
        print(f"\n❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\n💾 Checkpoint saved. You can resume later.")
        print("🔒 All caches preserved for next run.")


if __name__ == "__main__":
    main()

🎯 I3D-Processor-ScientificData v8.0
✨ SCIENTIFIC DATA READY - Production Version

🔧 CRITICAL FIXES:
   ✅ Audio features: 94 (verified with assert)
   ✅ MediaPipe landmarks: 478 face (468+10 iris) + 33 pose
   ✅ Explicit librosa parameters (n_fft, hop_length, etc)
   ✅ Deterministic STT mode available
   ✅ Per-file re-extraction flags (not global)
   ✅ Complete data dictionary generated
   ✅ Run manifest for full reproducibility
   ✅ Processing summary (per-file quality metrics)
   ✅ Decision logging (JSONL format)
   ✅ Environment capture (Python, packages, FFmpeg)
   ✅ SNR computation documented (STFT parameters)
   ✅ Sentiment analysis caveat documented
   ✅ Filename leakage warning
   ✅ Missing landmark policy: (0.0, 0.0, 0.0)
   ✅ Ethics & privacy statement included

📦 Standard Features:
   • Language: Indonesian
   • 📝 Manual Transcript Support (33 files)
   • Transcript Freezing (cache after first extraction)
   • Translation Freezing (MD5-based cache)
   • Indonesian number norm

C:\Users\user\AppData\Local\Temp\ipykernel_15680\2952079294.py:139: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2026-01-31 09:58:11,918 - [I3D] - INFO - ======================================================================
2026-01-31 09:58:11,918 - [I3D] - INFO - 🎯 I3D-Processor-ScientificData v8.0
2026-01-31 09:58:11,919 - [I3D] - INFO - ======================================================================
2026-01-31 09:58:11,919 - [I3D] - INFO - ✨ Scientific Data Ready Features:
2026-01-31 09:58:11,920 - [I3D] - INFO -    • Fixed 94 audio features (verified)
2026-01-31 09:58:11,920 - [I3D] - INFO -    • MediaPipe: 478 face (468+10 iris) + 33 pose
2026-01-31 09:58:11,921 - [I3D] - INFO -    • Runtime validation for landmark counts
2026-01-31 09:58:11,921 - [I3D] - I


✅ I3D DATASET PROCESSING COMPLETE!
📊 Total files: 1568
✓  Text extraction: 784/1568
📝 Manual transcripts: 27
💾 Cached transcripts: 614
✓  Numbers normalized: 234/311
✓  Translation: 784/784
🌐 Cached translations: 0
✓  Audio features (94): 784/1568
✓  Landmark extraction: 784/1568
👁️  Iris detection: 267,083 frames
🔄 Re-extractions: 0 triggered, 0 successful
⚠️  Failed samples: 0

📦 Output directory: f:\MULAI LAGI\eKSTRAKSI - FULL DATA\dataset\processed\I3D
📋 Detailed logs: f:\MULAI LAGI\eKSTRAKSI - FULL DATA\dataset\_logs\I3D
💾 Cache directory: f:\MULAI LAGI\eKSTRAKSI - FULL DATA\dataset\_cache\I3D
📊 Decision log: f:\MULAI LAGI\eKSTRAKSI - FULL DATA\dataset\_logs\I3D\decisions
📚 Metadata: f:\MULAI LAGI\eKSTRAKSI - FULL DATA\dataset\metadata\I3D

🔒 Deterministic Processing Summary:
   • Transcript cache: 614/784 from cache
   • Translation cache: 0/784 from cache
   • Manual transcripts: 27 files
   • Re-extractions: 0/0 successful

📈 Cache Efficiency: 78.3%
   ✓  Good cache utilizatio